In [ ]:
import numpy as np
import scipy as sp
from scipy import constants
import matplotlib.pyplot as plt
import sympy as sym
from sympy import I, exp, init_printing
from fractions import Fraction
from matplotlib.pyplot import figure
init_printing()

In [ ]:
Pin, Tmod, RL, Vpi, G_EM, Gopt, Rpd = sym.symbols('P_in T_mod R_L V_pi G_EM G_opt R_pd', real=True)
Sld, G_DM, F_DM = sym.symbols('S_LD G_DM, F_DM')
F_EM, Ipd, q, F, k, T, RIN = sym.symbols('F_EM I_pd q F k T RIN')

In [ ]:
### External Modulated Link Loss equation
sym.Eq(G_EM,(1/4)*((sym.pi*Pin*Tmod*RL)/(2*Vpi))*Gopt**2*Rpd**2)

In [ ]:
### External modulated noise figure equation
sym.Eq(F_EM,1+1/Gopt+(q*Pin*RL)/(2*Gopt*Rpd*k*T)+(RIN*Pin**2*RL)/(4*Gopt*Rpd**2*k*T))

In [ ]:
### Directly Modulated Link Loss
sym.Eq(G_DM, (1/4)*Sld**2*Gopt**2*Rpd**2)

In [ ]:
### Directly Modulated Noise Figure
sym.Eq(F_DM, 1+4/(Sld*Gopt*Rpd)**2+(2*q*Pin*RL)/(Sld**2*Gopt*Rpd*k*T)+(RIN*Pin**2*RL)/(Sld**2*k*T))


In [ ]:
# Parameter Definition 
Pin = np.linspace(0,25,251) #dBm
RIN = -160 #dB / Hz
Pin_lin = 10**(Pin/10)/1000 #W
RIN_lin = 10**(RIN/10) #/Hz

q = 1.6e-19
k = 1.38e-23
T = 290 #K
RL = 50 #Ohms
Rpd_Int = 1.09 #A/W
Rpd = 0.75 #A/W
Tmod = -4 #dB 
Tmod_lin = 10**(Tmod/10)
Vpi = 5 #V
Coupler = -0.5 #dB
VOA = -1.5 #dB
TDL = -1 #dB
ECoupler = -1.4 #dB
PC = -0.1 #dB
MRR_Weight = -0.2 #dB
MRR_Delay = -0.2 #dB
RFConnLoss = -0.25 #dB

#Typical from Charles Cox / ILX /
# http://www.cybel-llc.com/wp-content/uploads/2018/07/DM-DFB-Laser_030818a-v1.pdf
S_LD = 0.3761 #mW/mA
RIN_DML = -162 #dB
RIN_DML_lin = 10**(RIN_DML/10)

# Using IP3 = 35 at 150 mA calculate c
IP3_ref = 35
ID_ref = 150e-3 
c = ((2*S_LD)*10**(IP3_ref/10))/((150e-3/Rpd)**2*RL) 

# DML_IIP3 = 10*np.log10((c*Pin_lin**2*RL)/(2*S_LD))



### Balanced
Balanced_Gain = 3 #dB
RIN_suppression = 30 #dB
RIN_bal = RIN - RIN_suppression #dB / Hz
RIN_bal_lin = 10**(RIN_bal/10) #/Hz


# https://www.rflambda.com/pdf/lownoiseamplifier/RLNA00M65GD.pdf

# LNA_G = 19.5 #dB
# LNA_NF = 4.5 #dB
# LNA_G_lin = 10**(LNA_G/10)
# LNA_NF_lin = 10**(LNA_NF/10)


# https://www.rflambda.com/pdf/lownoiseamplifier/RLNA05M54GA.pdf
LNA_G = 23 #dB
LNA_NF = 4 #dB
LNA_G_lin = 10**(LNA_G/10)
LNA_NF_lin = 10**(LNA_NF/10)

# Power into the waveguide
Pin_lin_wg = np.zeros((1,len(Pin)))

# Optical Power into the waveguide
Pin_lin_wg_ext = 10**((Pin+ECoupler+Tmod)/10)/1000
Pin_lin_wg_dir = 10**((Pin+ECoupler)/10)/1000   


# function that returns dP/dz for nonlinear loss
from scipy.integrate import odeint
def model(P,z):
    alpha1TE = 1.04 #   db/cm Linear Loss from AMF review paper
    alpha2TE = 32.6 #   db/cm/W
    alpha3TE = 2.04e3 # db/cm/W^2
    dPdz = -alpha1TE * P - alpha2TE * P**2 - alpha3TE * P**3
    return dPdz

In [ ]:
def adjust_lightness(color, amount=0.5):
    import matplotlib.colors as mc
    import colorsys
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    return colorsys.hls_to_rgb(c[0], max(0, min(1, amount * c[1])), c[2])

In [ ]:
fs=16
isSaving = True
place_text = True
dpi_res = 300 
color = plt.cm.magma(np.linspace(0.175, 0.8, 5))
                      
line_EM = "-"
line_DM = "--"
line_EM_LNA = ":"
line_DM_LNA = "-."
# color_BalHy = colors[3]

color_EM = color[0]
color_DM = color[2]


label_DM = 'Direct Modulation'
label_EM = 'External Modulation'
label_LNA = ' w/ LNA'
label_Si = 'Silicon '


line_w = 2.5
dot_size = 10

# RF Anaylsis

### Gain - Discrete

In [ ]:
### EM Gain, sweeping Vpi
# I want to define top and bottom by real modulators and then sweep between
Vpi_worst = 14.4 # cite: Samani, Alireza, et al. "A low-voltage 35-GHz silicon photonic modulator-enabled 112-Gb/s transmission system." IEEE Photonics Journal 7.3 (2015): 1-13.
Vpi_best = 1 #cite: Eltes, Felix, et al. "A BaTiO 3-based electro-optic Pockels modulator monolithically integrated on an advanced silicon photonics platform." Journal of Lightwave Technology 37.5 (2019): 1456-1462.
# Best:
Vpi_sweep = np.linspace(Vpi_best,Vpi_worst,5) #dBm
G_EM = np.zeros((len(Vpi_sweep),len(Pin)))
G_EM_lin = np.zeros((len(Vpi_sweep),len(Pin)))


Gopt_EM = -5 # Could be typical for coupling then a couple weights and delays
Gopt_EM_lin = 10**(Gopt_EM/10) 

# Link Loss
for i in range(len(Vpi_sweep)):
    G_EM_lin[i,:] = (1/4)*((np.pi*Pin_lin*Tmod_lin*RL)/(2*Vpi_sweep[i]))**2*Gopt_EM_lin**2*Rpd**2 ##lin
    G_EM[i,:] = 10*np.log10(G_EM_lin[i,:])

In [ ]:
# Plot Link Loss verse Optical power / Sweep Vpi
plt.figure(figsize=(7,5))
color_adj_start = 1
color_increment = -0.4
for i in range(len(Vpi_sweep)):
    plt.plot(Pin,G_EM[i,:], color = adjust_lightness(color_EM,color_adj_start-i*color_increment), linestyle = line_EM, linewidth=line_w,label=label_EM)
#     if i == 0:
#         plt.legend(handlelength=3, loc='lower right')

plt.ylabel('Link Loss (dB)', fontsize=fs)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)

text_index = 15
best_vpi_str = 'Vπ = ' + str(Vpi_best) 
worst_vpi_str = 'Vπ = ' + str(Vpi_worst)

if place_text:
    plt.text(text_index, -10, best_vpi_str, fontsize = 12)
    plt.text(text_index, -50, worst_vpi_str, fontsize = 12)
    plt.arrow(16.5, -13, 0, -30, width=0.1, shape="full", head_length = 3, head_width = 0.5, color = 'k',zorder=40)

plt.xlim(0,25)
# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
# plt.legend(handlelength=3, bbox_to_anchor=(1.57, .8),loc='right')
# plt.legend(handlelength=3, loc='lower right')
if isSaving:
    outputFile = 'TFig01_LL_EM.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res, bbox_inches='tight')
plt.show()


In [ ]:
print(G_EM[0,151])
print(G_EM[-1,151])

In [ ]:
### DM Gain, sweeping SLD
# I want to define top and bottom by real DML and then sweep between
Sld_worst = .0614 # cite: https://ieeexplore.ieee.org/abstract/document/4663579?casa_token=VuN6dqYBB4IAAAAA:kg-GQF3L2wE5-fRisoQxAkACxCGIj24KFYfYVHqYjTn8xQWCi02xMLWAEjuBDKp2jn95RkJB
Sld_best = .4 #cite: Sun, Wei, et al. "Design of 1.3-$\mu\text {m} $ High-Performance Directly Modulated Lasers Based on High-Order Slotted Surface Gratings." IEEE Journal of Quantum Electronics 53.5 (2017): 1-9.
# Best:
Sld_sweep = np.linspace(Sld_best,Sld_worst,5) #dBm
G_DM = np.zeros((len(Sld_sweep),len(Pin)))
G_DM_lin = np.zeros((len(Sld_sweep),len(Pin)))

Gopt_DM = -5 # Could be typical for coupling then a couple weights and delays
Gopt_DM_lin = 10**(Gopt_DM/10) 

# Link Loss
for i in range(len(Sld_sweep)):
    G_DM_lin[i,:] = np.full((1, len(Pin)),(1/4)*(Sld_sweep[i]**2*Gopt_DM_lin**2*Rpd**2))##lin
    G_DM[i,:] = 10*np.log10(G_DM_lin[i,:]) ##dB

In [ ]:
# Plot Link Loss verse Optical power / Sweep SLD
plt.figure(figsize=(7,5))
color_adj_start = 1
color_increment = -0.2
for i in range(len(Sld_sweep)):
    plt.plot(Pin,G_DM[i,:], color = adjust_lightness(color_DM,color_adj_start-i*color_increment),linestyle = line_DM, linewidth=line_w,label=label_DM)
    if i == 0:
        plt.legend(handlelength=3, loc='lower right')
        
plt.ylabel('Link Loss (dB)', fontsize=fs)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)

text_index = 5
best_sld_str = 'Sld = ' + str(Sld_best) 
worst_sld_str = 'Sld = ' + str(Sld_worst)

if place_text:
    plt.text(text_index, -22, best_sld_str, fontsize = 12)
    plt.text(text_index, -47, worst_sld_str, fontsize = 12)
    plt.arrow(text_index+1.5, -23, 0, -21, width=0.1, shape="full", head_length = 1.5, head_width = 0.5, color = 'k',zorder=40)
plt.xlim(0,25)
plt.ylim(-50,-10)
# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
# plt.legend(handlelength=3, bbox_to_anchor=(1.57, .8),loc='right')
# plt.legend(handlelength=3, loc='lower right')
if isSaving:
    outputFile = 'TFig02_LL_DM.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res, bbox_inches='tight')
plt.show()


In [ ]:
print(np.max(G_DM[0,:]))
print(np.max(G_DM[-1,:]))

In [ ]:
fig = plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
color_adj_start = 1
color_increment = -0.4
for i in range(len(Vpi_sweep)):
    plt.plot(Pin,G_EM[i,:], color = adjust_lightness(color_EM,color_adj_start-i*color_increment), linestyle = line_EM, linewidth=line_w,label=label_EM)
    if i == 0:
        plt.legend(handlelength=3, loc='lower right')

plt.ylabel('Link Loss (dB)', fontsize=fs)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)

text_index = 15
best_vpi_str = 'Vπ = ' + str(Vpi_best) 
worst_vpi_str = 'Vπ = ' + str(Vpi_worst)

if place_text:
    plt.text(text_index, -10, best_vpi_str, fontsize = 12)
    plt.text(text_index, -50, worst_vpi_str, fontsize = 12)
    plt.arrow(16.5, -13, 0, -30, width=0.1, shape="full", head_length = 3, head_width = 0.5, color = 'k',zorder=40)

plt.xlim(0,25)
plt.ylim(-70,0)
# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')

plt.subplot(1, 2, 2)
color_adj_start = 1
color_increment = -0.2
for i in range(len(Sld_sweep)):
    plt.plot(Pin,G_DM[i,:], color = adjust_lightness(color_DM,color_adj_start-i*color_increment),linestyle = line_DM, linewidth=line_w,label=label_DM)
    if i == 0:
        plt.legend(handlelength=3, loc='lower right')
        
plt.ylabel('Link Loss (dB)', fontsize=fs)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)

text_index = 5
best_sld_str = 'Sld = ' + str(Sld_best) 
worst_sld_str = 'Sld = ' + str(Sld_worst)

if place_text:
    plt.text(text_index, -22, best_sld_str, fontsize = 12)
    plt.text(text_index, -47, worst_sld_str, fontsize = 12)
    plt.arrow(text_index+1.5, -23, 0, -21, width=0.1, shape="full", head_length = 1.5, head_width = 0.5, color = 'k',zorder=40)
plt.xlim(0,25)
plt.ylim(-70,0)
# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
plt.legend(handlelength=3, bbox_to_anchor=(1.57, .8),loc='right')
plt.legend(handlelength=3, loc='lower right')
if isSaving:
    outputFile = 'TFig0102_LL.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res, bbox_inches='tight')
plt.show()

In [ ]:
### Plot both sweeps of discrete link loss together
# Plot Link Loss verse Optical power / Sweep SLD
plt.figure(figsize=(7,5))
color_adj_start = 1
color_increment = -0.4
for i in range(len(Vpi_sweep)):
    if i == 0:
        plt.plot(Pin,G_EM[i,:], color = adjust_lightness(color_EM,color_adj_start-i*color_increment), linestyle = line_EM, linewidth=line_w,label=label_EM)
    else: 
        plt.plot(Pin,G_EM[i,:], color = adjust_lightness(color_EM,color_adj_start-i*color_increment), linestyle = line_EM, linewidth=line_w, label='_nolegend_')

i=0
color_adj_start = 1
color_increment = -0.2
for i in range(len(Sld_sweep)):
    if i == 0:
        plt.plot(Pin,G_DM[i,:], color = adjust_lightness(color_DM,color_adj_start-i*color_increment), linestyle = line_DM, linewidth=line_w,label=label_DM)
    else: 
        plt.plot(Pin,G_DM[i,:], color = adjust_lightness(color_DM,color_adj_start-i*color_increment), linestyle = line_DM, linewidth=line_w, label='_nolegend_')

        
plt.ylabel('Link Loss (dB)', fontsize=fs)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)



plt.xlim(0,25)
# plt.ylim(-50,20)
# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
# plt.legend(handlelength=3, bbox_to_anchor=(1.57, .8),loc='right')
plt.legend(handlelength=3, loc='lower right')
if isSaving:
    outputFile = 'TFig03_LL_EMDM.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res, bbox_inches='tight')
plt.show()


### Noise Figure

In [ ]:
# Noise Figure and Regions of Noise Dependency

In [ ]:
### EM NF
# Standard Link

#Typical from Charles Cox / ILX /
# http://www.cybel-llc.com/wp-content/uploads/2018/07/DM-DFB-Laser_030818a-v1.pdf
S_LD = 0.3761 #mW/mA
RIN_DML = -162 #dB
RIN_DML_lin = 10**(RIN_DML/10)


Pin2 = np.linspace(-10,25,126)
Pin2_lin = 10**(Pin2/10)/1000 #W
G_EM = np.zeros((1,len(Pin2)))
G_EM_lin = np.zeros((1,len(Pin2)))
NF_EM = np.zeros((1,len(Pin2)))
NF_EM_lin = np.zeros((1,len(Pin)))


Gopt_EM = -5 # Could be typical for coupling then a couple weights and delays
Gopt_EM_lin = 10**(Gopt_EM/10) 

# Link Loss

G_EM_lin = (1/4)*((np.pi*Pin2_lin*Tmod_lin*RL)/(2*Vpi))**2*Gopt_EM_lin**2*Rpd**2 ##lin
G_EM = 10*np.log10(G_EM_lin)

# Noise Figure
NF_EM_lin = 1 + \
    (16*Vpi**2)/(np.pi**2 * Pin2_lin**2 * Gopt_EM_lin **2 * Rpd **2 * RL **2) + \
    (16*q*Vpi**2)/(np.pi**2 * Pin2_lin * Gopt_EM_lin * Rpd * RL*k*T) + \
    (4*Vpi**2*RIN_lin)/(np.pi**2*RL*k*T)

NF_EM = 10*np.log10(NF_EM_lin) ##dB

In [ ]:
## EM NF
# Standard Link

G_DM = np.zeros((1,len(Pin2)))
G_DM_lin = np.zeros((1,len(Pin2)))
NF_DM = np.zeros((1,len(Pin2)))
NF_DM_lin = np.zeros((1,len(Pin2)))

Gopt_DM = -10 # Could be typical for coupling then a couple weights and delays
Gopt_DM_lin = 10**(Gopt_DM/10) 

#Noise Figure
G_DM_lin = np.full((1, len(Pin2)),(1/4)*(S_LD**2*Gopt_DM_lin**2*Rpd**2))##lin
G_DM = 10*np.log10(G_DM_lin) ##dB

F_DM_lin = 1+(4/((S_LD*Gopt_DM_lin*Rpd)**2)  + \
    (2*q*Pin2_lin*RL)/(S_LD**2*Gopt_DM_lin*Rpd*k*T)+ \
    (RIN_lin*Pin2_lin**2*RL)/(S_LD**2*k*T))

NF_DM = 10*np.log10(F_DM_lin) ##dB

In [ ]:
# Plot Standard NF for Noise Region Definition
plt.figure(figsize=(7,5))

plt.plot(Pin2,NF_EM, color =color_EM,linewidth=line_w,label=label_EM, linestyle = line_EM)
plt.plot(Pin2,NF_DM, color =color_DM,linewidth=line_w,label=label_DM, linestyle = line_DM)

plt.ylabel('Noise Figure (dB)', fontsize=fs)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)
# plt.title('RF Link Loss of Rx path for Canceller Architectures ', fontsize=15)



plt.xlim(0,25)
plt.ylim(30,65)
# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.legend(handlelength=3, loc='best')
plt.grid(b=None, which='major', axis='both')
if isSaving:
    outputFile = 'TFig04_NF_EMDM.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res)
plt.show()

In [ ]:
print(np.min(NF_DM))
print(np.min(NF_EM))

In [ ]:
# Plot Standard NF for Noise Region Definition
y_th = -2*Pin2+54
y_shot = -1*Pin2+52.5
y_rin = 0*Pin2+36.7

plt.figure(figsize=(7,5))

plt.plot(Pin2,NF_EM, color ='k',linewidth=line_w,label='_nolegend_', linestyle = line_EM)
plt.plot(Pin2[0:55], y_th[0:55], color = color[1], linestyle = "--", linewidth = line_w-0.5,label='Thermal')
plt.plot(Pin2, y_shot, color = color[2], linestyle = "--", linewidth = line_w-0.5,label='Shot')
plt.plot(Pin2[60:-1], y_rin[60:-1], color = color[3], linestyle = "--", linewidth = line_w-0.5,label='RIN')
# plt.vlines(4.1,0,50, color = 'k', linewidth = 0.7)
# plt.vlines(15.5,0,50, color = 'k', linewidth = 0.7)

plt.ylabel('Noise Figure (dB)', fontsize=fs)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)
# plt.title('RF Link Loss of Rx path for Canceller Architectures ', fontsize=15)



plt.xlim(-10,25)
# plt.ylim(35,57)
# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
plt.legend(handlelength=3, ncol =3, bbox_to_anchor=(0, 1.1), loc='upper left')
# ax.set_xticks([np.linspace(0,25,6)])
if isSaving:
    outputFile = 'TFig05_NFEM_Noise_Regions.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res)
plt.show()


In [ ]:
#Derivative on NF
derivative =  np.diff(NF_EM)/np.diff(Pin2)
plt.plot(Pin2[0:-1],derivative, color ='k',linewidth=line_w,label='_nolegend_', linestyle = line_EM)

plt.ylabel('Noise Figure Slope (dB)', fontsize=fs)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)
# plt.title('RF Link Loss of Rx path for Canceller Architectures ', fontsize=15)



plt.xlim(-10,25)
# plt.ylim(25,38)
# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
# plt.legend(handlelength=3, ncol =3, bbox_to_anchor=(0, 1.1), loc='upper left')
if isSaving:
    outputFile = 'TFig06_NFEM_Slope.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res)
plt.show()


In [ ]:
fig = plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
plt.plot(Pin2,NF_EM, color ='k',linewidth=line_w,label='_nolegend_', linestyle = line_EM)
plt.plot(Pin2[0:55], y_th[0:55], color = color[1], linestyle = "--", linewidth = line_w-0.5,label='Thermal')
plt.plot(Pin2, y_shot, color = color[2], linestyle = "--", linewidth = line_w-0.5,label='Shot')
plt.plot(Pin2[60:-1], y_rin[60:-1], color = color[3], linestyle = "--", linewidth = line_w-0.5,label='RIN')
# plt.vlines(4.1,0,50, color = 'k', linewidth = 0.7)
# plt.vlines(15.5,0,50, color = 'k', linewidth = 0.7)

plt.ylabel('Noise Figure (dB)', fontsize=fs)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)

plt.xlim(-10,25)
# plt.ylim(35,57)
# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
plt.legend(handlelength=3, ncol =3, bbox_to_anchor=(0, 1.1), loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(Pin2[0:-1],derivative, color ='k',linewidth=line_w,label='_nolegend_', linestyle = line_EM)

plt.ylabel('Noise Figure Slope (dB)', fontsize=fs)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)
plt.xlim(-10,25)
# plt.ylim(25,38)
# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
if isSaving:
    outputFile = 'TFig0506.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res)
plt.show()

In [ ]:
# DM and EM NF verse RIN

In [ ]:
###########################
#  RIN SWEEPS
###########################
RIN_best = -170 # cite: https://investor.emcore.com/static-files/67ec465c-1ad8-4277-acf1-ae6e42438f6e ||| https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9890135
RIN_worst = -150 #cite Leinse, A., et al. "Hybrid interconnection of InP and TriPleX photonic integrated circuits for new module functionality." Optical Interconnects XIX. Vol. 10924. SPIE, 2019.
RIN_sweep = np.linspace(-150,-175,5)
RIN_sweep_lin = [10**(x/10) for x in RIN_sweep]

G_EM = np.zeros((1,len(Pin)))
G_EM_lin = np.zeros((1,len(Pin)))
NF_EM = np.zeros((len(RIN_sweep),len(Pin)))
NF_EM_lin = np.zeros((len(RIN_sweep),len(Pin)))


Gopt_EM = -5 #dB
Gopt_EM_lin = 10**(Gopt_EM/10) 

# Link Loss
G_EM_lin = (1/4)*((np.pi*Pin_lin*Tmod_lin*RL)/(2*Vpi))**2*Gopt_EM_lin**2*Rpd**2 ##lin
G_EM = 10*np.log10(G_EM_lin) ##dB

# Noise Figure for loop with RIN sweep
for i in range(len(RIN_sweep)):
    NF_EM_lin[i,:] = 1 + \
        (16*Vpi**2)/(np.pi**2 * Pin_lin**2 * Gopt_EM_lin **2 * Rpd **2 * RL **2) + \
        (16*q*Vpi**2)/(np.pi**2 * Pin_lin * Gopt_EM_lin * Rpd * RL*k*T) + \
        (4*Vpi**2*RIN_sweep_lin[i])/(np.pi**2*RL*k*T)
    NF_EM[i,:] = 10*np.log10(NF_EM_lin[i,:]) ##dB

    
    
G_DM = np.zeros((1,len(Pin)))
G_DM_lin = np.zeros((1,len(Pin)))
NF_DM = np.zeros((len(RIN_sweep),len(Pin)))
NF_DM_lin = np.zeros((len(RIN_sweep),len(Pin)))


Gopt = 5
Gopt_lin = 10**(Gopt/10)

# Link Loss
G_DM_lin = np.full((1, len(Pin)),(1/4)*(S_LD**2*Gopt_lin**2*Rpd**2))##lin
G_DM = 10*np.log10(G_DM_lin) ##dB


# Noise Figure for loop with RIN sweep
for i in range(len(RIN_sweep)):
    NF_DM_lin[i,:] = 1+(4/((S_LD*Gopt_lin*Rpd)**2)  + \
                        (2*q*Pin_lin*RL)/(S_LD**2*Gopt_lin*Rpd*k*T) + \
                        (RIN_sweep_lin[i]*Pin_lin**2*RL)/(S_LD**2*k*T))
    NF_DM[i,:] = 10*np.log10(NF_DM_lin[i,:]) ##dB


In [ ]:
# Plot NF verse RIN sweep
plt.figure(figsize=(7,5))
color_adj_start1 = 1
color_adj_start2 = 1
color_increment1 = -.4
color_increment2 = -.2
# Noise Figure for loop with RIN sweep
for i in range(len(RIN_sweep)):
    if i == 0: 
        plt.plot(Pin,NF_EM[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label=label_EM)
#         plt.plot(Pin,NF_DM[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label=label_DM)
    else: 
        plt.plot(Pin,NF_EM[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label='_nolegend_')
#         plt.plot(Pin,NF_DM[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label='_nolegend_')


plt.ylabel('Noise Figure (dB)', fontsize=fs)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)
# plt.title('RF Link Loss of Rx path for Canceller Architectures ', fontsize=15)

text_index = 20
best_RIN_str = 'RIN = ' + str(RIN_best) 
worst_RIN_str = 'RIN = ' + str(RIN_worst)

if place_text:
    plt.text(text_index, 27, best_RIN_str, fontsize = 12)
    plt.text(text_index, 48, worst_RIN_str, fontsize = 12)
    plt.arrow(text_index-1, 30, 0, 18, width=0.1, shape="full", head_length = 1, head_width = 0.5, color = 'k',zorder=40)


plt.xlim(0,25)
# plt.ylim(20,50)
# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
# plt.legend(handlelength=3, bbox_to_anchor=(1.57, .8),loc='right')
plt.legend(handlelength=3, loc='lower left')
if isSaving:
    outputFile = 'TFig07_NFEM_RIN_Sweep.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res, bbox_inches='tight')
plt.show()
# 

In [ ]:
# Plot NF verse RIN sweep
plt.figure(figsize=(7,5))

# Noise Figure for loop with RIN sweep
for i in range(len(RIN_sweep)):
    if i == 0: 
        plt.plot(Pin,NF_EM[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label=label_EM)
        plt.plot(Pin,NF_DM[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label=label_DM)
    else: 
        plt.plot(Pin,NF_EM[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label='_nolegend_')
        plt.plot(Pin,NF_DM[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label='_nolegend_')

plt.ylabel('Noise Figure (dB)', fontsize=fs)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)
# plt.title('RF Link Loss of Rx path for Canceller Architectures ', fontsize=15)



plt.xlim(0,25)
# plt.ylim(20,50)
# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
# plt.legend(handlelength=3, bbox_to_anchor=(1.57, .8),loc='right')
plt.legend(handlelength=3, loc='lower right')
if isSaving:
    outputFile = 'TFig09_NFEMDM_RINSweep.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res, bbox_inches='tight')
plt.show()


In [ ]:
# Plot NF verse RIN sweep
plt.figure(figsize=(7,5))

# Noise Figure for loop with RIN sweep
for i in range(len(RIN_sweep)):
    if i == 0: 
#         plt.plot(Pin,NF_EM[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label=label_EM)
        plt.plot(Pin,NF_DM[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label=label_DM)
    else: 
#         plt.plot(Pin,NF_EM[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label='_nolegend_')
        plt.plot(Pin,NF_DM[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label='_nolegend_')


plt.ylabel('Noise Figure (dB)', fontsize=fs)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)
# plt.title('RF Link Loss of Rx path for Canceller Architectures ', fontsize=15)

text_index = 10
best_RIN_str = 'RIN = ' + str(RIN_best) 
worst_RIN_str = 'RIN = ' + str(RIN_worst)

if place_text:
    plt.text(text_index, 18, best_RIN_str, fontsize = 12)
    plt.text(text_index, 50, worst_RIN_str, fontsize = 12)
    plt.arrow(text_index-1, 17, 0, 30, width=0.1, shape="full", head_length = 1, head_width = 0.5, color = 'k',zorder=40)

plt.xlim(0,25)
# plt.ylim(20,50)
# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
# plt.legend(handlelength=3, bbox_to_anchor=(1.57, .8),loc='right')
plt.legend(handlelength=3, loc='best')
if isSaving:
    outputFile = 'TFig08_NFDM_RIN_sweep.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res, bbox_inches='tight')
plt.show()


In [ ]:
# Plot NF verse RIN sweep
plt.figure(figsize=(7,5))

# Noise Figure for loop with RIN sweep
for i in range(len(RIN_sweep)):
    if i == 0: 
#         plt.plot(Pin,NF_EM[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label=label_EM)
        plt.plot(Pin,NF_DM[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label=label_DM)
    else: 
#         plt.plot(Pin,NF_EM[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label='_nolegend_')
        plt.plot(Pin,NF_DM[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label='_nolegend_')

plt.ylabel('Noise Figure (dB)', fontsize=fs)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)
# plt.title('RF Link Loss of Rx path for Canceller Architectures ', fontsize=15)



plt.xlim(0,10)
plt.ylim(10,40)
# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
# plt.legend(handlelength=3, bbox_to_anchor=(1.57, .8),loc='right')
plt.legend(handlelength=3, loc='best')
# outputFile = 'TFig_DM_RIN_swp.pdf'
# plt.savefig(outputFile, format='pdf', dpi=1000, bbox_inches='tight')
plt.show()


### Low Noise Amplification

In [ ]:
# https://www.rflambda.com/pdf/lownoiseamplifier/RLNA05M54GA.pdf
LNA_G = 23 #dB
LNA_NF = 4 #dB
LNA_G_lin = 10**(LNA_G/10)
LNA_NF_lin = 10**(LNA_NF/10)

In [ ]:
fig = plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
color_adj_start1 = 1
color_adj_start2 = 1
color_increment1 = -.4
color_increment2 = -.2
# Noise Figure for loop with RIN sweep
for i in range(len(RIN_sweep)):
    if i == 0: 
        plt.plot(Pin,NF_EM[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label=label_EM)
#         plt.plot(Pin,NF_DM[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label=label_DM)
    else: 
        plt.plot(Pin,NF_EM[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label='_nolegend_')
#         plt.plot(Pin,NF_DM[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label='_nolegend_')


plt.ylabel('Noise Figure (dB)', fontsize=fs)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)
# plt.title('RF Link Loss of Rx path for Canceller Architectures ', fontsize=15)

text_index = 20
best_RIN_str = 'RIN = ' + str(RIN_best) 
worst_RIN_str = 'RIN = ' + str(RIN_worst)

if place_text:
    plt.text(text_index, 27, best_RIN_str, fontsize = 12)
    plt.text(text_index, 48, worst_RIN_str, fontsize = 12)
    plt.arrow(text_index-1, 30, 0, 18, width=0.1, shape="full", head_length = 1, head_width = 0.5, color = 'k',zorder=40)


plt.xlim(0,25)
# plt.ylim(20,50)
# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
# plt.legend(handlelength=3, bbox_to_anchor=(1.57, .8),loc='right')
plt.legend(handlelength=3, loc='lower left')

plt.subplot(1, 2, 2)
# Noise Figure for loop with RIN sweep
for i in range(len(RIN_sweep)):
    if i == 0: 
#         plt.plot(Pin,NF_EM[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label=label_EM)
        plt.plot(Pin,NF_DM[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label=label_DM)
    else: 
#         plt.plot(Pin,NF_EM[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label='_nolegend_')
        plt.plot(Pin,NF_DM[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label='_nolegend_')


plt.ylabel('Noise Figure (dB)', fontsize=fs)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)
# plt.title('RF Link Loss of Rx path for Canceller Architectures ', fontsize=15)

text_index = 10
best_RIN_str = 'RIN = ' + str(RIN_best) 
worst_RIN_str = 'RIN = ' + str(RIN_worst)

if place_text:
    plt.text(text_index, 18, best_RIN_str, fontsize = 12)
    plt.text(text_index, 50, worst_RIN_str, fontsize = 12)
    plt.arrow(text_index-1, 17, 0, 30, width=0.1, shape="full", head_length = 1, head_width = 0.5, color = 'k',zorder=40)

plt.xlim(0,25)
# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
# plt.legend(handlelength=3, bbox_to_anchor=(1.57, .8),loc='right')
plt.legend(handlelength=3, loc='best')
if isSaving:
    outputFile = 'TFig0708_NFDM_RIN_sweep.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res, bbox_inches='tight')
plt.show()


### Low Noise Amplification

In [ ]:
# https://www.rflambda.com/pdf/lownoiseamplifier/RLNA05M54GA.pdf
LNA_G = 23 #dB
LNA_NF = 4 #dB
LNA_G_lin = 10**(LNA_G/10)
LNA_NF_lin = 10**(LNA_NF/10)

In [ ]:
### EM and DM, single link comparison 
# Standard Link

G_EM = np.zeros((1,len(Pin)))
G_EM_lin = np.zeros((1,len(Pin)))
NF_EM = np.zeros((1,len(Pin)))
NF_EM_lin = np.zeros((1,len(Pin)))
G_EM_LNA = np.zeros((1,len(Pin)))
G_EM_lin_LNA = np.zeros((1,len(Pin)))
NF_EM_LNA = np.zeros((1,len(Pin)))
NF_EM_lin_LNA = np.zeros((1,len(Pin)))


Gopt_EM = -5 # Could be typical for coupling then a couple weights and delays
Gopt_EM_lin = 10**(Gopt_EM/10) 

# Link Loss
G_EM_lin = (1/4)*((np.pi*Pin_lin*Tmod_lin*RL)/(2*Vpi))**2*Gopt_EM_lin**2*Rpd**2 ##lin
G_EM = 10*np.log10(G_EM_lin)

# Noise Figure
NF_EM_lin = 1 + \
    (16*Vpi**2)/(np.pi**2 * Pin_lin**2 * Gopt_EM_lin **2 * Rpd **2 * RL **2) + \
    (16*q*Vpi**2)/(np.pi**2 * Pin_lin * Gopt_EM_lin * Rpd * RL*k*T) + \
    (4*Vpi**2*RIN_lin)/(np.pi**2*RL*k*T)
NF_EM = 10*np.log10(NF_EM_lin) ##dB

#LNA
G_EM_LNA = G_EM + LNA_G
#LNA Friis
NF_EM_lin_LNA = LNA_NF_lin+(NF_EM_lin-1)/LNA_G_lin
NF_EM_LNA = 10*np.log10(NF_EM_lin_LNA) ##dB



In [ ]:
G_DM = np.zeros((1,len(Pin)))
G_DM_lin = np.zeros((1,len(Pin)))
G_DM_LNA = np.zeros((1,len(Pin)))
NF_DM = np.zeros((1,len(Pin)))
NF_DM_LNA = np.zeros((1,len(Pin)))
NF_DM_lin = np.zeros((1,len(Pin)))
NF_DM_lin_LNA = np.zeros((1,len(Pin)))

Gopt = -5#dB

#convert to linear
Gopt_lin = 10**(Gopt/10)
Pin_lin = 10**(Pin/10)/1000
RIN_lin = 10**(RIN/10)

# Link Loss
G_DM_lin = np.full((1, len(Pin)),(1/4)*(S_LD**2*Gopt_lin**2*Rpd**2))##lin
G_DM = 10*np.log10(G_DM_lin) ##dB

#Noise Figure
F_DM_lin = 1+(4/((S_LD*Gopt_lin*Rpd)**2)  + \
    (2*q*Pin_lin*RL)/(S_LD**2*Gopt_lin*Rpd*k*T)+ \
    (RIN_lin*Pin_lin**2*RL)/(S_LD**2*k*T))
NF_DM = 10*np.log10(F_DM_lin) ##dB

G_DM_LNA = G_DM + LNA_G

#LNA Friis
F_DM_lin_LNA = LNA_NF_lin+(F_DM_lin-1)/LNA_G_lin
NF_DM_LNA = 10*np.log10(F_DM_lin_LNA) ##dB


In [ ]:
# Plot Standard NF for Noise Region Definition
plt.figure(figsize=(7,5))

plt.plot(Pin,NF_EM, color =color_EM,linewidth=line_w,label=label_EM, linestyle = line_EM)
plt.plot(Pin,NF_DM, color =color_DM,linewidth=line_w,label=label_DM, linestyle = line_DM)
plt.plot(Pin,NF_EM_LNA, color =color_EM,linewidth=line_w,label=label_EM+label_LNA, linestyle = line_EM_LNA)
plt.plot(Pin,NF_DM_LNA, color =color_DM,linewidth=line_w,label=label_DM+label_LNA, linestyle = line_DM_LNA)

plt.ylabel('Noise Figure (dB)', fontsize=fs)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)
# plt.title('RF Link Loss of Rx path for Canceller Architectures ', fontsize=15)



plt.xlim(0,25)
# plt.ylim(30,50)
# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
plt.legend(handlelength=3, loc='best')
if isSaving:
    outputFile = 'TFig10_EMDM_wLNA.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res)
plt.show()


In [ ]:
print(np.min(NF_DM_LNA))
print(np.min(NF_EM_LNA))
print(np.min(NF_DM)-np.min(NF_DM_LNA))
print(np.min(NF_EM)-np.min(NF_EM_LNA))

In [ ]:
### EM and DM, Sweep LNA Gain
# Standard Link
LNA_G_best = 30
LNA_G_worst = 15

LNA_G_sweep = np.linspace(LNA_G_worst,LNA_G_best,5)
LNA_G_lin_sweep = [10**(x/10) for x in LNA_G_sweep]

G_EM = np.zeros((1,len(Pin)))
G_EM_lin = np.zeros((1,len(Pin)))
NF_EM = np.zeros((1,len(Pin)))
NF_EM_lin = np.zeros((1,len(Pin)))
G_EM_LNA = np.zeros((len(LNA_G_lin_sweep),len(Pin)))
G_EM_lin_LNA = np.zeros((len(LNA_G_lin_sweep),len(Pin)))
NF_EM_LNA = np.zeros((len(LNA_G_lin_sweep),len(Pin)))
NF_EM_lin_LNA = np.zeros((len(LNA_G_lin_sweep),len(Pin)))


Gopt_EM = -5 # Could be typical for coupling then a couple weights and delays
Gopt_EM_lin = 10**(Gopt_EM/10) 

# Link Loss
G_EM_lin = (1/4)*((np.pi*Pin_lin*Tmod_lin*RL)/(2*Vpi))**2*Gopt_EM_lin**2*Rpd**2 ##lin
G_EM = 10*np.log10(G_EM_lin)

# Noise Figure
NF_EM_lin = 1 + \
    (16*Vpi**2)/(np.pi**2 * Pin_lin**2 * Gopt_EM_lin **2 * Rpd **2 * RL **2) + \
    (16*q*Vpi**2)/(np.pi**2 * Pin_lin * Gopt_EM_lin * Rpd * RL*k*T) + \
    (4*Vpi**2*RIN_lin)/(np.pi**2*RL*k*T)
NF_EM = 10*np.log10(NF_EM_lin) ##dB

#LNA
for i in range(len(LNA_G_sweep)):
    G_EM_LNA[i,:] = G_EM + LNA_G_sweep[i]
    #LNA Friis
    NF_EM_lin_LNA[i,:] = LNA_NF_lin+(NF_EM_lin-1)/LNA_G_lin_sweep[i]
    NF_EM_LNA[i,:] = 10*np.log10(NF_EM_lin_LNA[i,:]) ##dB


In [ ]:
G_DM = np.zeros((1,len(Pin)))
G_DM_lin = np.zeros((1,len(Pin)))
G_DM_LNA = np.zeros((1,len(Pin)))
NF_DM = np.zeros((1,len(Pin)))
G_DM_LNA = np.zeros((len(LNA_G_lin_sweep),len(Pin)))
G_DM_lin_LNA = np.zeros((len(LNA_G_lin_sweep),len(Pin)))
NF_DM_LNA = np.zeros((len(LNA_G_lin_sweep),len(Pin)))
NF_DM_lin_LNA = np.zeros((len(LNA_G_lin_sweep),len(Pin)))

Gopt_EM = -5 #dB

#convert to linear
Gopt_lin = 10**(Gopt/10)
Pin_lin = 10**(Pin/10)/1000
RIN_lin = 10**(RIN/10)

# Link Loss
G_DM_lin = np.full((1, len(Pin)),(1/4)*(S_LD**2*Gopt_lin**2*Rpd**2))##lin
G_DM = 10*np.log10(G_DM_lin) ##dB

#Noise Figure
NF_DM_lin = 1+(4/((S_LD*Gopt_lin*Rpd)**2)  + \
    (2*q*Pin_lin*RL)/(S_LD**2*Gopt_lin*Rpd*k*T)+ \
    (RIN_lin*Pin_lin**2*RL)/(S_LD**2*k*T))
NF_DM = 10*np.log10(NF_DM_lin) ##dB


#LNA
for i in range(len(LNA_G_sweep)):
    
    G_DM_LNA = G_DM + LNA_G_sweep[i]
    #LNA Friis
    NF_DM_lin_LNA[i,:] = LNA_NF_lin+(NF_DM_lin-1)/LNA_G_lin_sweep[i]
    NF_DM_LNA[i,:] = 10*np.log10(NF_DM_lin_LNA[i,:]) ##dB


In [ ]:
### EM and DM, Sweep LNA NF
plt.figure(figsize=(7,5))
color_adj_start1 = 1
color_adj_start2 = 1
color_increment1 = -.4
color_increment2 = -.2
# Noise Figure for loop with RIN sweep
for i in range(len(LNA_G_sweep)):
    if i == 0: 
        plt.plot(Pin,NF_EM_LNA[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label=label_EM)
#         plt.plot(Pin,NF_DM_LNA[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label=label_DM)
    else: 
        plt.plot(Pin,NF_EM_LNA[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label='_nolegend_')
#         plt.plot(Pin,NF_DM_LNA[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label='_nolegend_')


plt.ylabel('Noise Figure (dB)', fontsize=fs)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)
# plt.title('RF Link Loss of Rx path for Canceller Architectures ', fontsize=15)



plt.xlim(0,25)
# plt.ylim(20,50)
# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
# plt.legend(handlelength=3, bbox_to_anchor=(1.57, .8),loc='right')
plt.legend(handlelength=3, loc='best')
# outputFile = 'TFig_DM_RIN_swp.pdf'
# plt.savefig(outputFile, format='pdf', dpi=1000, bbox_inches='tight')
plt.show()
# 

In [ ]:
### EM and DM, Sweep LNA NF
plt.figure(figsize=(7,5))
color_adj_start1 = 1
color_adj_start2 = 1
color_increment1 = -.4
color_increment2 = -.2
# Noise Figure for loop with RIN sweep
for i in range(len(LNA_G_sweep)):
    if i == 0: 
        plt.plot(Pin,NF_EM_LNA[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label=label_EM)
        plt.plot(Pin,NF_DM_LNA[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label=label_DM)
    else: 
        plt.plot(Pin,NF_EM_LNA[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label='_nolegend_')
        plt.plot(Pin,NF_DM_LNA[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label='_nolegend_')


plt.ylabel('Noise Figure (dB)', fontsize=fs)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)
# plt.title('RF Link Loss of Rx path for Canceller Architectures ', fontsize=15)

text_index = 12
best_LNA_G_str = 'LNA G = ' + str(LNA_G_best) 
worst_LNA_G_str = 'LNA G = ' + str(LNA_G_worst)

if place_text:
    plt.text(text_index, 5, best_LNA_G_str, fontsize = 12)
    plt.text(text_index, 30, worst_LNA_G_str, fontsize = 12)
    plt.arrow(text_index-0.5, 30, 0, -25, width=0.05, shape="full", head_length = 1.5, head_width = 0.5, color = 'k',zorder=40)

plt.xlim(0,25)

# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
# plt.legend(handlelength=3, bbox_to_anchor=(1.57, .8),loc='right')
plt.legend(handlelength=3, loc='upper center')
if isSaving:
    outputFile = 'TFig11_EMDM_LNA_G_sweep.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res, bbox_inches='tight')
plt.show()
# 

In [ ]:
### EM and DM, Sweep LNA NF
plt.figure(figsize=(7,5))
color_adj_start1 = 1
color_adj_start2 = 1
color_increment1 = -.4
color_increment2 = -.2
# Noise Figure for loop with RIN sweep
for i in range(len(LNA_G_sweep)):
    if i == 0: 
#         plt.plot(Pin,NF_EM_LNA[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label=label_EM)
        plt.plot(Pin,NF_DM_LNA[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label=label_DM)
    else: 
#         plt.plot(Pin,NF_EM_LNA[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label='_nolegend_')
        plt.plot(Pin,NF_DM_LNA[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label='_nolegend_')


plt.ylabel('Noise Figure (dB)', fontsize=fs)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)
# plt.title('RF Link Loss of Rx path for Canceller Architectures ', fontsize=15)



plt.xlim(0,25)

# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
# plt.legend(handlelength=3, bbox_to_anchor=(1.57, .8),loc='right')
plt.legend(handlelength=3, loc='best')
# outputFile = 'TFig_DM_RIN_swp.pdf'
# plt.savefig(outputFile, format='pdf', dpi=1000, bbox_inches='tight')
plt.show()
# 

In [ ]:
### EM and DM, Sweep LNA NF
# Standard Link
LNA_NF_best = 3
LNA_NF_worst = 10

LNA_NF_sweep = np.linspace(LNA_NF_worst,LNA_NF_best,5)
LNA_NF_lin_sweep = [10**(x/10) for x in LNA_NF_sweep]

G_EM = np.zeros((1,len(Pin)))
G_EM_lin = np.zeros((1,len(Pin)))
NF_EM = np.zeros((1,len(Pin)))
NF_EM_lin = np.zeros((1,len(Pin)))
G_EM_LNA = np.zeros((len(LNA_NF_sweep),len(Pin)))
G_EM_lin_LNA = np.zeros((len(LNA_NF_sweep),len(Pin)))
NF_EM_LNA = np.zeros((len(LNA_NF_sweep),len(Pin)))
NF_EM_lin_LNA = np.zeros((len(LNA_NF_sweep),len(Pin)))


Gopt_EM = -10 # Could be typical for coupling then a couple weights and delays
Gopt_EM_lin = 10**(Gopt_EM/10) 

# Link Loss
G_EM_lin = (1/4)*((np.pi*Pin_lin*Tmod_lin*RL)/(2*Vpi))**2*Gopt_EM_lin**2*Rpd**2 ##lin
G_EM = 10*np.log10(G_EM_lin)

# Noise Figure
NF_EM_lin = 1 + \
    (16*Vpi**2)/(np.pi**2 * Pin_lin**2 * Gopt_EM_lin **2 * Rpd **2 * RL **2) + \
    (16*q*Vpi**2)/(np.pi**2 * Pin_lin * Gopt_EM_lin * Rpd * RL*k*T) + \
    (4*Vpi**2*RIN_lin)/(np.pi**2*RL*k*T)
NF_EM = 10*np.log10(NF_EM_lin) ##dB

#LNA
G_EM_LNA = G_EM + LNA_G
for i in range(len(LNA_NF_sweep)):
    #LNA Friis
    NF_EM_lin_LNA[i,:] = LNA_NF_lin_sweep[i]+(NF_EM_lin-1)/LNA_G_lin
    NF_EM_LNA[i,:] = 10*np.log10(NF_EM_lin_LNA[i,:]) ##dB


In [ ]:
G_DM = np.zeros((1,len(Pin)))
G_DM_lin = np.zeros((1,len(Pin)))
G_DM_LNA = np.zeros((1,len(Pin)))
NF_DM = np.zeros((1,len(Pin)))
G_DM_LNA = np.zeros((len(LNA_NF_sweep),len(Pin)))
G_DM_lin_LNA = np.zeros((len(LNA_NF_sweep),len(Pin)))
NF_DM_LNA = np.zeros((len(LNA_NF_sweep),len(Pin)))
NF_DM_lin_LNA = np.zeros((len(LNA_NF_sweep),len(Pin)))

Gopt_EM = -5 #dB

#convert to linear
Gopt_lin = 10**(Gopt/10)
Pin_lin = 10**(Pin/10)/1000
RIN_lin = 10**(RIN/10)

# Link Loss
G_DM_lin = np.full((1, len(Pin)),(1/4)*(S_LD**2*Gopt_lin**2*Rpd**2))##lin
G_DM = 10*np.log10(G_DM_lin) ##dB

#Noise Figure
NF_DM_lin = 1+(4/((S_LD*Gopt_lin*Rpd)**2)  + \
    (2*q*Pin_lin*RL)/(S_LD**2*Gopt_lin*Rpd*k*T)+ \
    (RIN_lin*Pin_lin**2*RL)/(S_LD**2*k*T))
NF_DM = 10*np.log10(NF_DM_lin) ##dB


#LNA
G_DM_LNA = G_DM + LNA_G
for i in range(len(LNA_NF_sweep)):    
    #LNA Friis
    NF_DM_lin_LNA[i,:] = LNA_NF_lin_sweep[i]+(NF_DM_lin-1)/LNA_G_lin
    NF_DM_LNA[i,:] = 10*np.log10(NF_DM_lin_LNA[i,:]) ##dB


In [ ]:
### EM and DM, Sweep LNA NF
plt.figure(figsize=(7,5))
color_adj_start1 = 1
color_adj_start2 = 1
color_increment1 = -.4
color_increment2 = -.2
# Noise Figure for loop with RIN sweep
for i in range(len(LNA_NF_sweep)):
    if i == 0: 
        plt.plot(Pin,NF_EM_LNA[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label=label_EM)
        plt.plot(Pin,NF_DM_LNA[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label=label_DM)
    else: 
        plt.plot(Pin,NF_EM_LNA[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label='_nolegend_')
        plt.plot(Pin,NF_DM_LNA[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label='_nolegend_')


plt.ylabel('Noise Figure (dB)', fontsize=fs)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)
# plt.title('RF Link Loss of Rx path for Canceller Architectures ', fontsize=15)

text_index = 2
text_index2 = 20
best_LNA_NF_str = 'LNA NF = ' + str(LNA_NF_best) 
worst_LNA_NF_str = 'LNA NF = ' + str(LNA_NF_worst)

if place_text:
    plt.text(text_index, 6, best_LNA_NF_str, fontsize = 12)
    plt.text(text_index, 13, worst_LNA_NF_str, fontsize = 12)
    plt.arrow(text_index-0.5, 6, 0, 7, width=0.05, shape="full", head_length = 1, head_width = 0.5, color = 'k',zorder=40)
 
    plt.text(text_index2, 12, best_LNA_NF_str, fontsize = 12)
    plt.text(text_index2, 21, worst_LNA_NF_str, fontsize = 12)
    plt.arrow(text_index2-0.5, 13, 0, 7, width=0.05, shape="full", head_length = 1, head_width = 0.5, color = 'k',zorder=40)

    
plt.xlim(0,25)

plt.xlim(0,25)

# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
# plt.legend(handlelength=3, bbox_to_anchor=(1.57, .8),loc='right')
plt.legend(handlelength=3, loc='best')
if isSaving:
    outputFile = 'TFig12_EMDM_LNA_NF_sweep.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res, bbox_inches='tight')
plt.show()
# 

### Linear Nonlinear waveguide loss

In [ ]:
# Nonlinearities in Silicon Waveguide
# Tokushima, Masatoshi, Jun Ushida, and Takahiro Nakamura. "Nonlinear loss characterization 
# of continuous wave guiding in silicon wire waveguides." Applied Physics Express 14.12 (2021):
# 122008.

# 350 nm (H) × 200 nm (V ). The upper cladding was
# a 2 μm thick silicon dioxide layer,

BetaTPA = 24.8 #cm/GW
# TE
alpha1TE = 1.04 #   db/cm Linear Loss from AMF review paper
alpha2TE = 32.6 #   db/cm/W
alpha3TE = 2.04e3 # db/cm/W^2

# # TM
# alpha1TM = 3.72 #   db/cm Linear Loss 
# alpha2TM = 1.45 #   db/cm/W
# alpha3TM = 4.72e2 # db/cm/W^2


WgL = 0.1 # cm / 10 mm
Pin_nl = np.linspace(15,25,5)
Pin_lin_nl = 10**(Pin_nl/10)/1000

# distance points points
Wgz = np.linspace(0,0.1,31) #Waveguide distance

# initial condition
# y0 = .02 #W

#Nonlinear loss|
NL_loss = np.zeros((len(Pin_nl),1))

plt.figure(figsize=(7,5))
i=0
for y0 in Pin_lin_nl:
    # solve ODE
    Pin_z = odeint(model,y0,Wgz)
    NL_loss[i] = 10*np.log10(Pin_z[-1]/Pin_z[0])
    # plot results
    plt.plot(Wgz*10,10*np.log10(Pin_z/1e-3), color= color[i], linewidth=line_w, label = 'Popt: '+ str(Pin_nl[i])+ 'dBm')
    i+=1
plt.xlabel('Waveguide Distance (mm)', fontsize=12)
plt.ylabel('Optical Power (dBm)', fontsize=12)
plt.grid(b=None, which='major', axis='both')
plt.legend(handlelength=3, loc='best')
plt.xlim(0,1)
if isSaving:
    outputFile = 'TFig13_OptPow_vs_Wg.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res, bbox_inches='tight')
plt.show()



In [ ]:
fig = plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
WgL = 0.1 # cm / 10 mm
Pin_nl = np.linspace(15,25,5)
Pin_lin_nl = 10**(Pin_nl/10)/1000

# distance points points
Wgz = np.linspace(0,0.1,31) #Waveguide distance

# initial condition
# y0 = .02 #W

#Nonlinear loss|
NL_loss = np.zeros((len(Pin_nl),1))

i=0
for y0 in Pin_lin_nl:
    # solve ODE
    Pin_z = odeint(model,y0,Wgz)
    NL_loss[i] = 10*np.log10(Pin_z[-1]/Pin_z[0])
    # plot results
    plt.plot(Wgz*10,10*np.log10(Pin_z/1e-3), color= color[i], linewidth=line_w, label = 'Popt: '+ str(Pin_nl[i])+ 'dBm')
    i+=1
plt.xlabel('Waveguide Distance (mm)', fontsize=12)
plt.ylabel('Optical Power (dBm)', fontsize=12)
plt.grid(b=None, which='major', axis='both')
plt.legend(handlelength=3, loc='best')
plt.xlim(0,1)
WgL = 0.1 # cm / 10 mm
Pin_nl = np.linspace(0,25,51)
Pin_lin_nl = 10**(Pin_nl/10)/1000

# distance points points
Wgz = np.linspace(0,WgL,31) #Waveguide distance


#Nonlinear loss|
NL_loss = np.zeros((len(Pin_nl),1))

i=0
for y0 in Pin_lin_nl:
    # solve ODE
    Pin_z = odeint(model,y0,Wgz)
    NL_loss[i] = 10*np.log10(Pin_z[-1]/Pin_z[0])
    # plot results
    i+=1


plt.subplot(1, 2, 2)
plt.plot(Pin_nl,NL_loss, color = 'k', linewidth = line_w)
plt.xlabel('Power In (dBm)', fontsize=12)
plt.ylabel('Linear and Nonlinear Loss (dB)', fontsize=12)
plt.ylabel('Optical Waveguide Loss over 1 mm (dB)', fontsize=12)
plt.xlim(0,25)
plt.ylim(-10,0)
plt.grid(b=None, which='major', axis='both')
if isSaving:
    outputFile = 'TFig1314.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res, bbox_inches='tight')
plt.show()

In [ ]:
WgL = 0.1 # cm / 10 mm
Pin_nl = np.linspace(0,25,51)
Pin_lin_nl = 10**(Pin_nl/10)/1000

# distance points points
Wgz = np.linspace(0,WgL,31) #Waveguide distance


#Nonlinear loss|
NL_loss = np.zeros((len(Pin_nl),1))

i=0
for y0 in Pin_lin_nl:
    # solve ODE
    Pin_z = odeint(model,y0,Wgz)
    NL_loss[i] = 10*np.log10(Pin_z[-1]/Pin_z[0])
    # plot results
    i+=1



plt.figure(figsize=(7,5))
plt.plot(Pin_nl,NL_loss, color = 'k', linewidth = line_w)
plt.xlabel('Power In (dBm)', fontsize=12)
plt.ylabel('Linear and Nonlinear Loss (dB)', fontsize=12)
plt.ylabel('Optical Waveguide Loss over 1 mm (dB)', fontsize=12)
plt.xlim(0,25)
plt.ylim(-10,0)
plt.grid(b=None, which='major', axis='both')
if isSaving:
    outputFile = 'TFig14_NLwgLoss_OptPowIn.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res, bbox_inches='tight')
plt.show()

### RF Performance Impact of Nonlinear Waveguide loss

In [ ]:
#compute link loss for lowest power link 
#init
G_EM = np.zeros((1,len(Pin)))
G_EM_LNA = np.zeros((1,len(Pin)))
G_EM_lin = np.zeros((1,len(Pin)))
NF_EM = np.zeros((1,len(Pin)))
NF_EM_LNA = np.zeros((1,len(Pin)))
NF_EM_lin = np.zeros((1,len(Pin)))
NF_EM_lin_LNA = np.zeros((1,len(Pin)))
Gopt_nonlin = np.linspace(0,0,len(Pin))


Pin_lin_wg = 10**((Pin+ECoupler)/10)/1000

WgL = .2 # cm
Wgz = np.linspace(0,WgL,11) #Waveguide distance

#Nonlinear loss
i=0
for y0 in Pin_lin_wg:
    # solve ODE
    Pin_z = odeint(model,y0,Wgz)
    Gopt_nonlin[i] = 10*np.log10(Pin_z[-1]/Pin_z[0])
    i+=1
Gopt = -2+Gopt_nonlin

#convert to linear
Gopt_lin = 10**(Gopt/10)


# Link Loss
G_EM_lin = (1/4)*((np.pi*Pin_lin*Tmod_lin*RL)/(2*Vpi))**2*Gopt_lin**2*Rpd**2 ##lin
G_EM = 10*np.log10(G_EM_lin) ##dB
G_EM_LNA = G_EM + LNA_G

# Noise Figure
NF_EM_lin = 1 + \
    (16*Vpi**2)/(np.pi**2 * Pin_lin**2 * Gopt_lin **2 * Rpd **2 * RL **2) + \
    (16*q*Vpi**2)/(np.pi**2 * Pin_lin * Gopt_lin * Rpd * RL*k*T) + \
    (4*Vpi**2*RIN_lin)/(np.pi**2*RL*k*T)
NF_EM = 10*np.log10(NF_EM_lin) ##dB

#LNA Friis
NF_EM_lin_LNA = LNA_NF_lin+(NF_EM_lin-1)/LNA_G_lin
NF_EM_LNA = 10*np.log10(NF_EM_lin_LNA) ##dB


In [ ]:
G_DM_Int = np.zeros((1,len(Pin)))
G_DM_Int_lin = np.zeros((1,len(Pin)))
G_DM_Int_LNA = np.zeros((1,len(Pin)))
NF_DM_Int = np.zeros((1,len(Pin)))
NF_DM_Int_LNA = np.zeros((1,len(Pin)))
NF_DM_Int_lin = np.zeros((1,len(Pin)))
NF_DM_Int_lin_LNA = np.zeros((1,len(Pin)))
Gopt_nonlin_DM_Int = np.linspace(0,0,len(Pin))


WgL = .2 # cm
Wgz = np.linspace(0,WgL,len(Pin)) #Waveguide distance
Pin_lin_wg = 10**((Pin+ECoupler)/10)/1000
len(Pin)
#Nonlinear loss
i=0
for y0 in Pin_lin_wg_ext:
    # solve ODE
    Pin_z = odeint(model,y0,Wgz)
    Gopt_nonlin_DM_Int[i] = 10*np.log10(Pin_z[-1]/Pin_z[0])
    i+=1
# print(Gopt_nonlin_EM_Int)


Gopt = -2+Gopt_nonlin_DM_Int #dB
#convert to linear
Gopt_lin = 10**(Gopt/10)


# Link Loss
G_DM_Int_lin = np.full((1, len(Pin)),(1/4)*(S_LD**2*Gopt_lin**2*Rpd_Int**2))##lin
G_DM_Int = 10*np.log10(G_DM_Int_lin) ##dB

#Noise Figure
NF_DM_Int_lin = 1+(4/((S_LD*Gopt_lin*Rpd_Int)**2)  + \
    (2*q*Pin_lin*RL)/(S_LD**2*Gopt_lin*Rpd_Int*k*T)+ \
    (RIN_lin*Pin_lin**2*RL)/(S_LD**2*k*T))
NF_DM_Int = 10*np.log10(NF_DM_Int_lin) ##dB


In [ ]:
# Plot Standard NF for Noise Region Definition
plt.figure(figsize=(7,5))

plt.plot(Pin,G_EM, color =color_EM,linewidth=line_w,label=label_EM, linestyle = line_EM)
plt.plot(Pin,G_DM_Int[0], color =color_DM,linewidth=line_w,label=label_DM, linestyle = line_DM)

plt.ylabel('Link Loss (dB)', fontsize=fs)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)
# plt.title('RF Link Loss of Rx path for Canceller Architectures ', fontsize=15)



plt.xlim(0,25)
# plt.ylim(30
# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
plt.legend(handlelength=3, loc='best')
if isSaving:
    outputFile = 'TFig15_LL_EMDM_wWgLoss.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res)
plt.show()


In [ ]:
print(np.max(G_DM_Int[0]))
print(np.max(G_EM))

In [ ]:
# Plot Standard NF for Noise Region Definition
plt.figure(figsize=(7,5))

plt.plot(Pin,NF_EM, color =color_EM,linewidth=line_w,label=label_EM, linestyle = line_EM)
plt.plot(Pin,NF_DM_Int, color =color_DM,linewidth=line_w,label=label_DM, linestyle = line_DM)

plt.ylabel('Noise Figure (dB)', fontsize=fs)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)
# plt.title('RF Link Loss of Rx path for Canceller Architectures ', fontsize=15)



plt.xlim(0,25)
# plt.ylim(30
# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
plt.legend(handlelength=3, loc='best')
if isSaving:
    outputFile = 'TFig16_NF_EMDM_wWgLoss.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res)
plt.show()


In [ ]:
print(np.min(NF_DM_Int))
print(np.min(NF_EM))

In [ ]:
fig = plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
plt.plot(Pin,G_EM, color =color_EM,linewidth=line_w,label=label_EM, linestyle = line_EM)
plt.plot(Pin,G_DM_Int[0], color =color_DM,linewidth=line_w,label=label_DM, linestyle = line_DM)

plt.ylabel('Link Loss (dB)', fontsize=fs)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)
plt.xlim(0,25)
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
plt.legend(handlelength=3, loc='best')

plt.subplot(1, 2, 2)
plt.plot(Pin,NF_EM, color =color_EM,linewidth=line_w,label=label_EM, linestyle = line_EM)
plt.plot(Pin,NF_DM_Int, color =color_DM,linewidth=line_w,label=label_DM, linestyle = line_DM)

plt.ylabel('Noise Figure (dB)', fontsize=fs)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)
plt.xlim(0,25)
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
plt.legend(handlelength=3, loc='best')
plt.tight_layout()

if isSaving:
    outputFile = 'TFig1516.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res)
    
plt.show()

### Nonlinear loss of resonator

In [ ]:
# from https://arxiv.org/pdf/1507.00686.pdf
# Typical Build up = 17.3
RefBuildup = 17.3
# Multiply incoming light by that intensity over length of ring
# Nik BSS rings are radii 22 μm 
r_bssmrr = 15
c_bssmrr = 2*np.pi*r_bssmrr

In [ ]:
####### Nonlinear MRR Loss 
Pin_nl = np.linspace(10,20,5)
WgL = 0.1 # cm / 10 mm
Pin_lin_wg_mrr = (10**((Pin_nl+ECoupler)/10)/1000)*RefBuildup
# distance points points
WgL = c_bssmrr*1e-4 # cm
# distance points points
Wgz = np.linspace(0,WgL,31) #Waveguide distance


#Nonlinear loss
Gopt_nonlin_mrr = np.linspace(0,0,len(Pin_nl))

plt.figure(figsize=(7,5))
i=0
for y0 in Pin_lin_wg_mrr:
    # solve ODE
    Pin_z = odeint(model,y0,Wgz)
    Gopt_nonlin_mrr[i] = 10*np.log10(Pin_z[-1]/Pin_z[0])
    # plot results
    plt.plot(Wgz,10*np.log10(Pin_z/1e-3), color= color[i], linewidth=line_w, label = 'Popt: '+ str(Pin_nl[i])+ 'dBm')
    print(np.max(10*np.log10(Pin_z/1e-3)))
    i+=1
plt.xlabel('Waveguide Distance (mm)', fontsize=12)
plt.ylabel('Optical Power within Ring(dBm)', fontsize=12)
plt.grid(b=None, which='major', axis='both')
plt.legend(handlelength=3, loc='best')
plt.xlim(0,WgL)
if isSaving:
    outputFile = 'TFig17_OptPow_vs_MRRloss.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res)
plt.show()



In [ ]:
fig = plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
Pin_nl = np.linspace(10,20,5)
WgL = 0.1 # cm / 10 mm
Pin_lin_wg_mrr = (10**((Pin_nl+ECoupler)/10)/1000)*RefBuildup
# distance points points
WgL = c_bssmrr*1e-4 # cm
# distance points points
Wgz = np.linspace(0,WgL,31) #Waveguide distance


#Nonlinear loss
Gopt_nonlin_mrr = np.linspace(0,0,len(Pin_nl))

i=0
for y0 in Pin_lin_wg_mrr:
    # solve ODE
    Pin_z = odeint(model,y0,Wgz)
    Gopt_nonlin_mrr[i] = 10*np.log10(Pin_z[-1]/Pin_z[0])
    # plot results
    plt.plot(Wgz,10*np.log10(Pin_z/1e-3), color= color[i], linewidth=line_w, label = 'Popt: '+ str(Pin_nl[i])+ 'dBm')
    i+=1
plt.xlabel('Waveguide Distance (mm)', fontsize=12)
plt.ylabel('Optical Power within Ring(dBm)', fontsize=12)
plt.grid(b=None, which='major', axis='both')
plt.legend(handlelength=3, loc='best')
plt.xlim(0,WgL)

plt.subplot(1, 2, 2)
WgL = 0.1 # cm / 10 mm
Pin_nl = np.linspace(0,25,51)
Pin_lin_nl = 10**(Pin_nl/10)/1000

# distance points points
Wgz = np.linspace(0,WgL,31) #Waveguide distance


#Nonlinear loss|
NL_loss = np.zeros((len(Pin_nl),1))

i=0
for y0 in Pin_lin_nl:
    # solve ODE
    Pin_z = odeint(model,y0,Wgz)
    NL_loss[i] = 10*np.log10(Pin_z[-1]/Pin_z[0])
    # plot results
    i+=1


### MRR
Pin_nl = np.linspace(0,25,51)
Pin_lin_wg_mrr = (10**((Pin_nl+ECoupler)/10)/1000)*RefBuildup

# distance points points
WgL = c_bssmrr*1e-4 # cm
Wgz = np.linspace(0,WgL,31) #Waveguide distance


#Nonlinear loss|
Gopt_nonlin_mrr = np.zeros((len(Pin_nl),1))

i=0
for y0 in Pin_lin_wg_mrr:
    # solve ODE
    Pin_z = odeint(model,y0,Wgz)
    Gopt_nonlin_mrr[i] = 10*np.log10(Pin_z[-1]/Pin_z[0])
    # plot results
    i+=1

plt.plot(Pin_nl,Gopt_nonlin_mrr, color = 'k', linestyle=':',linewidth = line_w, label = 'MRR Loss')
plt.plot(Pin_nl,NL_loss, color = 'k', linewidth = line_w, label = '1 mm Waveguide Loss')
plt.xlabel('Power In (dBm)', fontsize=12)
plt.ylabel('Optical Loss (dB)', fontsize=12)
plt.xlim(0,25)
plt.legend(handlelength=3, loc='best')
# plt.ylim(-10,0)
plt.grid(b=None, which='major', axis='both')
plt.tight_layout()

if isSaving:
    outputFile = 'TFig1718.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res)
plt.show()

In [ ]:
WgL = 0.1 # cm / 10 mm
Pin_nl = np.linspace(0,25,51)
Pin_lin_nl = 10**(Pin_nl/10)/1000

# distance points points
Wgz = np.linspace(0,WgL,31) #Waveguide distance


#Nonlinear loss|
NL_loss = np.zeros((len(Pin_nl),1))

i=0
for y0 in Pin_lin_nl:
    # solve ODE
    Pin_z = odeint(model,y0,Wgz)
    NL_loss[i] = 10*np.log10(Pin_z[-1]/Pin_z[0])
    # plot results
    i+=1





### MRR
Pin_nl = np.linspace(0,25,51)
Pin_lin_wg_mrr = (10**((Pin_nl+ECoupler)/10)/1000)*RefBuildup

# distance points points
WgL = c_bssmrr*1e-4 # cm
Wgz = np.linspace(0,WgL,31) #Waveguide distance


#Nonlinear loss|
Gopt_nonlin_mrr = np.zeros((len(Pin_nl),1))

i=0
for y0 in Pin_lin_wg_mrr:
    # solve ODE
    Pin_z = odeint(model,y0,Wgz)
    Gopt_nonlin_mrr[i] = 10*np.log10(Pin_z[-1]/Pin_z[0])
    # plot results
    i+=1

plt.figure(figsize=(7,5))
plt.plot(Pin_nl,Gopt_nonlin_mrr, color = 'k', linestyle=':',linewidth = line_w, label = 'MRR Loss')
plt.plot(Pin_nl,NL_loss, color = 'k', linewidth = line_w, label = '1 mm Waveguide Loss')
plt.xlabel('Power In (dBm)', fontsize=12)
plt.ylabel('Optical Loss (dB)', fontsize=12)
plt.xlim(0,25)
plt.legend(handlelength=3, loc='best')
# plt.ylim(-10,0)
plt.grid(b=None, which='major', axis='both')
if isSaving:
    outputFile = 'TFig18_NLmrrLoss_OptPowIn.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res)
plt.show()

In [ ]:
### Sweep Buildup
Buildup_best =  4.7358 #cite RF An of PNN
Buildup_worst = 40 #find citation 
Buildup_sweep = np.linspace(Buildup_worst,Buildup_best,5)

Pin_nl = np.linspace(0,25,51)
Pin_lin_wg_mrr_sweep = [(10**((Pin_nl+ECoupler)/10)/1000)*x for x in Buildup_sweep]

# distance points points
WgL = c_bssmrr*1e-4 # cm
Wgz = np.linspace(0,WgL,31) #Waveguide distance


#Nonlinear loss|
Gopt_nonlin_mrr = np.zeros((len(Buildup_sweep),len(Pin_nl)))

for j in range(len(Buildup_sweep)):
    i=0
    for y0 in Pin_lin_wg_mrr_sweep[j]:
        # solve ODE
        Pin_z = odeint(model,y0,Wgz)
        Gopt_nonlin_mrr[j,i] = 10*np.log10(Pin_z[-1]/Pin_z[0])
        # plot results
        i+=1

In [ ]:
### EM and DM, Sweep LNA NF
plt.figure(figsize=(7,5))
color_adj_start1 = 1
color_adj_start2 = 1
color_increment1 = -.4
color_increment2 = -.2

for i in range(len(Buildup_sweep)):
    if i == 0: 
        plt.plot(Pin_nl,Gopt_nonlin_mrr[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle=':', linewidth=line_w,label='MRR Loss')
    else: 
        plt.plot(Pin_nl,Gopt_nonlin_mrr[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle=':', linewidth=line_w,label='_nolegend_')

text_index = 19
best_Bup_str = 'Buildup = ' + str(np.round(Buildup_best,2)) 
worst_Bup_str = 'Buildup = ' + str(Buildup_worst)

if place_text:
    plt.text(text_index, -2.5, best_Bup_str, fontsize = 12)
    plt.text(text_index, -17.5, worst_Bup_str, fontsize = 12)
    plt.arrow(text_index+3, -3, 0, -13, width=0.05, shape="full", head_length = 0.5, head_width = 0.5, color = 'k',zorder=40)

        
plt.xlabel('Power In (dBm)', fontsize=12)
plt.ylabel('Optical Loss (dB)', fontsize=12)
plt.xlim(0,25)
plt.legend(handlelength=3, loc='best')
# plt.ylim(-10,0)
plt.grid(b=None, which='major', axis='both')
if isSaving:
    outputFile = 'TFig19_NLmrrLoss_Buildup_sweep.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res)
plt.show()


In [ ]:
G_EM = np.zeros((1,len(Pin)))
G_EM_LNA = np.zeros((1,len(Pin)))
G_EM_lin = np.zeros((1,len(Pin)))
NF_EM = np.zeros((1,len(Pin)))
NF_EM_LNA = np.zeros((1,len(Pin)))
NF_EM_lin = np.zeros((1,len(Pin)))
NF_EM_lin_LNA = np.zeros((1,len(Pin)))
SFDR3 = np.zeros((1,len(Pin)))
SFDR3_lin = np.zeros((1,len(Pin)))
SFDR_EM_LNA = np.zeros((1,len(Pin)))

ODEres = 100 

##############
#Nonlinear WG ODE off-chip mod

Pin_lin_wg = 10**((Pin+ECoupler)/10)/1000

#############
#Waveguide Length
WgL = 0.2 # cm
# distance points points
Wgz = np.linspace(0,WgL,ODEres) #Waveguide distance

#Nonlinear loss
Gopt_nonlin = np.linspace(0,0,len(Pin))
i=0
for y0 in Pin_lin_wg:
    # solve ODE
    Pin_z = odeint(model,y0,Wgz)
    Gopt_nonlin[i] = 10*np.log10(Pin_z[-1]/Pin_z[0])
    i+=1


####### Nonlinear MRR Loss 
Pin_lin_wg_mrr = (10**((Pin+ECoupler+Tmod)/10)/1000)*RefBuildup

#############
#Waveguide Length
WgL = c_bssmrr*1e-4 # cm
# distance points points
Wgz = np.linspace(0,WgL,ODEres) #Waveguide distance

#Nonlinear loss
Gopt_nonlin_mrr = np.linspace(0,0,len(Pin))
i=0
for y0 in Pin_lin_wg_mrr:
    # solve ODE
    Pin_z = odeint(model,y0,Wgz)
    Gopt_nonlin_mrr[i] = 10*np.log10(Pin_z[-1]/Pin_z[0])
    i+=1
    

Gopt = -2+Gopt_nonlin+Gopt_nonlin_mrr #dB

#convert to linear
Gopt_lin = 10**(Gopt/10)
Tmod_lin = 10**(Tmod/10)


# Link Loss
G_EM_lin = (1/4)*((np.pi*Pin_lin*Tmod_lin*RL)/(2*Vpi))**2*Gopt_lin**2*Rpd**2 ##lin
G_EM = 10*np.log10(G_EM_lin) ##dB
G_EM_LNA = G_EM + LNA_G

# Noise Figure
NF_EM_lin = 1 + \
    (16*Vpi**2)/(np.pi**2 * Pin_lin**2 * Gopt_lin **2 * Rpd **2 * RL **2) + \
    (16*q*Vpi**2)/(np.pi**2 * Pin_lin * Gopt_lin * Rpd * RL*k*T) + \
    (4*Vpi**2*RIN_lin)/(np.pi**2*RL*k*T)
NF_EM = 10*np.log10(NF_EM_lin) ##dB

#LNA Friis
NF_EM_lin_LNA = LNA_NF_lin+(NF_EM_lin-1)/LNA_G_lin
NF_EM_LNA = 10*np.log10(NF_EM_lin_LNA) ##dB

# SFDR
SFDR_3 = (2/3)*((4*Vpi**2)/(np.pi**2*RL) - NF_EM + 174 ) 
SFDR_EM_LNA = SFDR_3 + (NF_EM-NF_EM_LNA)-LNA_G


In [ ]:
G_DM = np.zeros((1,len(Pin)))
G_DM_LNA = np.zeros((1,len(Pin)))
G_DM_lin = np.zeros((1,len(Pin)))
NF_DM = np.zeros((1,len(Pin)))
NF_DM_LNA = np.zeros((1,len(Pin)))
NF_DM_lin_LNA = np.zeros((1,len(Pin)))
F_DM_lin = np.zeros((1,len(Pin)))

#Typical from Charles Cox / ILX /
# Consider APIC Linear DFB Laser
# http://www.cybel-llc.com/wp-content/uploads/2018/07/DM-DFB-Laser_030818a-v1.pdf

WgL = 0.2 # cm
Wgz = np.linspace(0,WgL,21) #Waveguide distance

#Nonlinear loss
Gopt_nonlin = np.linspace(0,0,len(Pin))
Pin_lin_wg = (10**((Pin+ECoupler)/10)/1000)*RefBuildup
i=0
for y0 in Pin_lin_wg:
    # solve ODE
    Pin_z = odeint(model,y0,Wgz)
    Gopt_nonlin[i] = 10*np.log10(Pin_z[-1]/Pin_z[0])
    i+=1

####### Nonlinear MRR Loss 
Pin_lin_wg_mrr = (10**((Pin+ECoupler)/10)/1000)*RefBuildup

#############
#Waveguide Length
WgL = c_bssmrr*1e-4 # cm
# distance points points
Wgz = np.linspace(0,WgL,11) #Waveguide distance

#Nonlinear loss
Gopt_nonlin_mrr = np.linspace(0,0,len(Pin))
i=0
for y0 in Pin_lin_wg_mrr:
    # solve ODE
    Pin_z = odeint(model,y0,Wgz)
    Gopt_nonlin_mrr[i] = 10*np.log10(Pin_z[-1]/Pin_z[0])
    i+=1
    
    
Gopt = -2+Gopt_nonlin+Gopt_nonlin_mrr #dB

#convert to linear
Gopt_lin = 10**(Gopt/10)

# Link Loss
G_DM_lin = np.full((1, len(Pin)),(1/4)*(S_LD**2*Gopt_lin**2*Rpd**2))##lin
G_DM = 10*np.log10(G_DM_lin) ##dB
G_DM_LNA = G_DM + LNA_G


#Noise Figure
F_DM_lin = 1+(4/((S_LD*Gopt_lin*Rpd)**2)  + \
    (2*q*Pin_lin*RL)/(S_LD**2*Gopt_lin*Rpd*k*T)+ \
    (RIN_lin*Pin_lin**2*RL)/(S_LD**2*k*T))

NF_DM = 10*np.log10(F_DM_lin) ##dB

#LNA Friis
F_DM_lin_LNA = LNA_NF_lin+(F_DM_lin-1)/LNA_G_lin
NF_DM_LNA = 10*np.log10(F_DM_lin_LNA) ##dB


# SFDR for DML
DML_IIP3 = 10*np.log10((c*Pin_lin**2*RL)/(2*S_LD))
SFDR_3DML = (2/3)*(DML_IIP3 - NF_DM + 174 ) 
SFDR_DML_LNA = SFDR_3DML + (NF_DM-NF_DM_LNA)-LNA_G


In [ ]:
# Plot G for MRR analog photonic link 
plt.figure(figsize=(7,5))

plt.plot(Pin,G_EM, color =color_EM,linewidth=line_w,label=label_EM, linestyle = line_EM)
plt.plot(Pin,G_DM[0,:], color =color_DM,linewidth=line_w,label=label_DM, linestyle = line_DM)

plt.ylabel('Link Loss (dB)', fontsize=fs)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)
# plt.title('RF Link Loss of Rx path for Canceller Architectures ', fontsize=15)



plt.xlim(0,25)
# plt.ylim(30
# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
plt.legend(handlelength=3, loc='best')
if isSaving:
    outputFile = 'TFig20_LL_EMDM_wNonlinearloss.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res)
plt.show()


In [ ]:
# Plot NF for MRR analog photonic link 
plt.figure(figsize=(7,5))

plt.plot(Pin,NF_EM, color =color_EM,linewidth=line_w,label=label_EM, linestyle = line_EM)
plt.plot(Pin,NF_DM, color =color_DM,linewidth=line_w,label=label_DM, linestyle = line_DM)

plt.ylabel('Noise Figure (dB)', fontsize=fs)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)
# plt.title('RF Link Loss of Rx path for Canceller Architectures ', fontsize=15)



plt.xlim(0,25)
plt.ylim(20,70)
# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
plt.legend(handlelength=3, loc='best')
if isSaving:
    outputFile = 'TFig21_NF_EMDM_wNonlinearloss.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res)
plt.show()


In [ ]:
print(np.min(NF_DM))
print(np.min(NF_EM))
print(np.max(G_DM[0,:]))
print(np.max(G_EM))

In [ ]:
fig = plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
plt.plot(Pin,G_EM, color =color_EM,linewidth=line_w,label=label_EM, linestyle = line_EM)
plt.plot(Pin,G_DM[0,:], color =color_DM,linewidth=line_w,label=label_DM, linestyle = line_DM)

plt.ylabel('Link Loss (dB)', fontsize=fs)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)
# plt.title('RF Link Loss of Rx path for Canceller Architectures ', fontsize=15)



plt.xlim(0,25)
# plt.ylim(30
# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
plt.legend(handlelength=3, loc='best')
plt.subplot(1, 2, 2)
plt.plot(Pin,NF_EM, color =color_EM,linewidth=line_w,label=label_EM, linestyle = line_EM)
plt.plot(Pin,NF_DM, color =color_DM,linewidth=line_w,label=label_DM, linestyle = line_DM)

plt.ylabel('Noise Figure (dB)', fontsize=fs)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)
# plt.title('RF Link Loss of Rx path for Canceller Architectures ', fontsize=15)



plt.xlim(0,25)
plt.ylim(20,70)
# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
plt.legend(handlelength=3, loc='best')
plt.tight_layout()

if isSaving:
    outputFile = 'TFig2021.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res)
plt.show()


In [ ]:
# Plot NF for MRR analog photonic link 
plt.figure(figsize=(7,5))
plt.plot(Pin,SFDR_3, color =color_EM,linewidth=line_w,label=label_EM, linestyle = line_EM)
plt.plot(Pin,SFDR_3DML, color =color_DM,linewidth=line_w,label=label_DM, linestyle = line_DM)

plt.ylabel('Spurious-Free Dynamic Range (dB/Hz^2/3)', fontsize=fs-2)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)
# plt.title('RF Link Loss of Rx path for Canceller Architectures ', fontsize=15)

plt.vlines(10,105,95, color ='k', linestyles='dashed',linewidth=line_w)
plt.text(0.5, 102, 'Small Signal Modulation\n Region of Direct Mod.', fontsize = 12)
plt.arrow(9.5, 98, -3, 0, width=0.3, shape="full", head_length = 0.3, head_width = 1, color = 'k')

plt.xlim(0,25)
plt.ylim(80,105)
# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
plt.legend(handlelength=3, loc='best')
if isSaving:
    outputFile = 'TFig22_SFDR_EMDM_wNonlinearloss.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res)
plt.show()


In [ ]:
print(np.max(SFDR_3DML))
print(Pin[np.argmax(SFDR_3DML)])

In [ ]:
### sweep build up

In [ ]:
### Sweep Buildup
Buildup_best = 4.3 # RF Analysis of PNN
Buildup_worst = 40
Buildup_sweep = np.linspace(Buildup_worst,Buildup_best,5)

Gopt = np.zeros((len(Buildup_sweep),len(Pin)))
Gopt_lin = np.zeros((len(Buildup_sweep),len(Pin)))
G_EM = np.zeros((len(Buildup_sweep),len(Pin)))
G_EM_LNA = np.zeros((len(Buildup_sweep),len(Pin)))
G_EM_lin = np.zeros((len(Buildup_sweep),len(Pin)))
NF_EM = np.zeros((len(Buildup_sweep),len(Pin)))
NF_EM_LNA = np.zeros((len(Buildup_sweep),len(Pin)))
NF_EM_lin = np.zeros((len(Buildup_sweep),len(Pin)))
NF_EM_lin_LNA = np.zeros((len(Buildup_sweep),len(Pin)))
SFDR_3 = np.zeros((len(Buildup_sweep),len(Pin)))
SFDR_3_lin = np.zeros((len(Buildup_sweep),len(Pin)))
SFDR_EM_LNA = np.zeros((len(Buildup_sweep),len(Pin)))

ODEres = 100 

##############
#Nonlinear WG ODE off-chip mod

Pin_lin_wg = 10**((Pin+ECoupler)/10)/1000

#############
#Waveguide Length
WgL = 0.2 # cm
# distance points points
Wgz = np.linspace(0,WgL,ODEres) #Waveguide distance

#Nonlinear loss
Gopt_nonlin = np.linspace(0,0,len(Pin))
i=0
for y0 in Pin_lin_wg:
    # solve ODE
    Pin_z = odeint(model,y0,Wgz)
    Gopt_nonlin[i] = 10*np.log10(Pin_z[-1]/Pin_z[0])
    i+=1


# distance points points
WgL = c_bssmrr*1e-4 # cm
Wgz = np.linspace(0,WgL,31) #Waveguide distance

Pin_lin_wg_mrr_sweep = [(10**((Pin+ECoupler)/10)/1000)*x for x in Buildup_sweep]

#Nonlinear loss|
Gopt_nonlin_mrr = np.zeros((len(Buildup_sweep),len(Pin)))


for j in range(len(Buildup_sweep)):
    i=0
    for y0 in Pin_lin_wg_mrr_sweep[j]:
        # solve ODE
        Pin_z = odeint(model,y0,Wgz)
        Gopt_nonlin_mrr[j,i] = 10*np.log10(Pin_z[-1]/Pin_z[0])
        # plot results
        i+=1
    
for z in range(len(Buildup_sweep)):
    Gopt[z,:] = -2+Gopt_nonlin+Gopt_nonlin_mrr[z,:] #dB

    #convert to linear
    Gopt_lin[z,:]  = 10**(Gopt[z,:]/10)


    # Link Loss
    G_EM_lin[z,:]  = (1/4)*((np.pi*Pin_lin*Tmod_lin*RL)/(2*Vpi))**2*Gopt_lin[z,:]**2*Rpd**2 ##lin
    G_EM[z,:]  = 10*np.log10(G_EM_lin[z,:]) ##dB
    G_EM_LNA[z,:]  = G_EM[z,:] + LNA_G

    # Noise Figure
    NF_EM_lin[z,:] = 1 + \
        (16*Vpi**2)/(np.pi**2 * Pin_lin**2 * Gopt_lin[z,:]**2 * Rpd **2 * RL **2) + \
        (16*q*Vpi**2)/(np.pi**2 * Pin_lin * Gopt_lin[z,:] * Rpd * RL*k*T) + \
        (4*Vpi**2*RIN_lin)/(np.pi**2*RL*k*T)
    NF_EM[z,:]  = 10*np.log10(NF_EM_lin[z,:]) ##dB

    #LNA Friis
    NF_EM_lin_LNA[z,:]  = LNA_NF_lin+(NF_EM_lin[z,:]-1)/LNA_G_lin
    NF_EM_LNA[z,:]  = 10*np.log10(NF_EM_lin_LNA[z,:]) ##dB

    # SFDR
    SFDR_3[z,:]  = (2/3)*((4*Vpi**2)/(np.pi**2*RL) - NF_EM[z,:] + 174 ) 
    SFDR_EM_LNA[z,:]  = SFDR_3[z,:]  + (NF_EM[z,:] - NF_EM_LNA[z,:] )-LNA_G


In [ ]:
Gopt = np.zeros((len(Buildup_sweep),len(Pin)))
Gopt_lin = np.zeros((len(Buildup_sweep),len(Pin)))
G_DM = np.zeros((len(Buildup_sweep),len(Pin)))
G_DM_LNA = np.zeros((len(Buildup_sweep),len(Pin)))
G_DM_lin = np.zeros((len(Buildup_sweep),len(Pin)))
NF_DM = np.zeros((len(Buildup_sweep),len(Pin)))
NF_DM_LNA = np.zeros((len(Buildup_sweep),len(Pin)))
NF_DM_lin_LNA = np.zeros((len(Buildup_sweep),len(Pin)))
NF_DM_lin = np.zeros((len(Buildup_sweep),len(Pin)))
SFDR_3DML = np.zeros((len(Buildup_sweep),len(Pin)))
SFDR_3DML_lin = np.zeros((len(Buildup_sweep),len(Pin)))
SFDR_DML_LNA = np.zeros((len(Buildup_sweep),len(Pin)))


#Typical from Charles Cox / ILX /
# Consider APIC Linear DFB Laser
# http://www.cybel-llc.com/wp-content/uploads/2018/07/DM-DFB-Laser_030818a-v1.pdf
WgL = 0.2 # cm
Wgz = np.linspace(0,WgL,21) #Waveguide distance

#Nonlinear loss
Gopt_nonlin = np.linspace(0,0,len(Pin))
Pin_lin_wg = (10**((Pin+ECoupler)/10)/1000)*RefBuildup
i=0
for y0 in Pin_lin_wg:
    # solve ODE
    Pin_z = odeint(model,y0,Wgz)
    Gopt_nonlin[i] = 10*np.log10(Pin_z[-1]/Pin_z[0])
    i+=1

for j in range(len(Buildup_sweep)):
    i=0
    for y0 in Pin_lin_wg_mrr_sweep[j]:
        # solve ODE
        Pin_z = odeint(model,y0,Wgz)
        Gopt_nonlin_mrr[j,i] = 10*np.log10(Pin_z[-1]/Pin_z[0])
        # plot results
        i+=1
    
for z in range(len(Buildup_sweep)):
    Gopt[z,:] = -2+Gopt_nonlin+Gopt_nonlin_mrr[z,:] #dB

    #convert to linear
    Gopt_lin[z,:]  = 10**(Gopt[z,:]/10)

    # Link Loss
    G_DM_lin[z,:] = (1/4)*(S_LD**2*Gopt_lin[z,:]**2*Rpd**2) ##lin
    G_DM[z,:] = 10*np.log10(G_DM_lin[z,:]) ##dB
    G_DM_LNA[z,:] = G_DM[z,:] + LNA_G


    #Noise Figure
    NF_DM_lin[z,:] = 1+(4/((S_LD*Gopt_lin[z,:]*Rpd)**2)  + \
        (2*q*Pin_lin*RL)/(S_LD**2*Gopt_lin[z,:]*Rpd*k*T)+ \
        (RIN_lin*Pin_lin**2*RL)/(S_LD**2*k*T))

    NF_DM[z,:] = 10*np.log10(NF_DM_lin[z,:]) ##dB

    #LNA Friis
    NF_DM_lin_LNA[z,:] = LNA_NF_lin+(NF_DM_lin[z,:]-1)/LNA_G_lin
    NF_DM_LNA[z,:] = 10*np.log10(NF_DM_lin_LNA[z,:]) ##dB


    # SFDR for DML
    DML_IIP3 = 10*np.log10((c*Pin_lin**2*RL)/(2*S_LD))
    SFDR_3DML[z,:] = (2/3)*(DML_IIP3 - NF_DM[z,:] + 174 ) 
    SFDR_DML_LNA[z,:] = SFDR_3DML[z,:] + (NF_DM[z,:] - NF_DM_LNA[z,:])-LNA_G


In [ ]:

plt.figure(figsize=(7,5))
color_adj_start1 = 1
color_adj_start2 = 1
color_increment1 = -.4
color_increment2 = -.2
# Noise Figure for loop with RIN sweep
for i in range(len(Buildup_sweep)):
    if i == 0: 
        plt.plot(Pin,G_EM[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label=label_EM)
        plt.plot(Pin,G_DM[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label=label_DM)
    else: 
        plt.plot(Pin,G_EM[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label='_nolegend_')
        plt.plot(Pin,G_DM[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label='_nolegend_')


plt.ylabel('Link Loss (dB)', fontsize=fs)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)
# plt.title('RF Link Loss of Rx path for Canceller Architectures ', fontsize=15)

text_index = 0.5
text_index2 = 1
best_Bup_str = 'Buildup = ' + str(np.round(Buildup_best,2)) 
worst_Bup_str = 'Buildup = ' + str(Buildup_worst)

if place_text:
    plt.text(text_index, -45, best_Bup_str, fontsize = 12)
    plt.text(text_index, -65, worst_Bup_str, fontsize = 12)
    plt.arrow(text_index+3, -47, 0, -10, width=0.05, shape="full", head_length = 1, head_width = 0.5, color = 'k',zorder=40)
    
#     plt.text(text_index2, -21, best_Bup_str, fontsize = 12)
#     plt.text(text_index2, -35, worst_Bup_str, fontsize = 12)
    plt.arrow(text_index2, -25, 0, -9, width=0.05, shape="full", head_length = 1, head_width = 0.5, color = 'k',zorder=40)

plt.xlim(0,25)

# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
# plt.legend(handlelength=3, bbox_to_anchor=(1.57, .8),loc='right')
plt.legend(handlelength=3, loc='best')
if isSaving:
    outputFile = 'TFig23_LL_EMDM_wNonlinearloss_buildup_sweep.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res)
plt.show()
# 

In [ ]:

plt.figure(figsize=(7,5))
color_adj_start1 = 1
color_adj_start2 = 1
color_increment1 = -.4
color_increment2 = -.2
# Noise Figure for loop with RIN sweep
for i in range(len(Buildup_sweep)):
    if i == 0: 
        plt.plot(Pin,NF_EM[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label=label_EM)
        plt.plot(Pin,NF_DM[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label=label_DM)
    else: 
        plt.plot(Pin,NF_EM[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label='_nolegend_')
        plt.plot(Pin,NF_DM[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label='_nolegend_')


plt.ylabel('Noise Figure (dB)', fontsize=fs)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)
# plt.title('RF Link Loss of Rx path for Canceller Architectures ', fontsize=15)

text_index = 0.5
text_index2 = 1
best_Bup_str = 'Buildup = ' + str(np.round(Buildup_best,2)) 
worst_Bup_str = 'Buildup = ' + str(Buildup_worst)

if place_text:
    plt.text(text_index, 57, best_Bup_str, fontsize = 12)
    plt.text(text_index, 41, worst_Bup_str, fontsize = 12)
    plt.arrow(text_index+3, 55, 0, -10, width=0.05, shape="full", head_length = 1, head_width = 0.5, color = 'k',zorder=40)
    
#     plt.text(text_index2, 30, best_Bup_str, fontsize = 12)
#     plt.text(text_index2, 20, worst_Bup_str, fontsize = 12)
    plt.arrow(text_index2,35, 0, -9, width=0.05, shape="full", head_length = 1, head_width = 0.5, color = 'k',zorder=40)


plt.xlim(0,25)

# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
# plt.legend(handlelength=3, bbox_to_anchor=(1.57, .8),loc='right')
plt.legend(handlelength=3, loc='best')
if isSaving:
    outputFile = 'TFig24_NF_EMDM_wNonlinearloss_buildup_sweep.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res)
plt.show()
# 

In [ ]:
fig = plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
color_adj_start1 = 1
color_adj_start2 = 1
color_increment1 = -.4
color_increment2 = -.2
# Noise Figure for loop with RIN sweep
for i in range(len(Buildup_sweep)):
    if i == 0: 
        plt.plot(Pin,G_EM[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label=label_EM)
        plt.plot(Pin,G_DM[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label=label_DM)
    else: 
        plt.plot(Pin,G_EM[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label='_nolegend_')
        plt.plot(Pin,G_DM[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label='_nolegend_')


plt.ylabel('Link Loss (dB)', fontsize=fs)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)
# plt.title('RF Link Loss of Rx path for Canceller Architectures ', fontsize=15)

text_index = 0.5
text_index2 = 1
best_Bup_str = 'Buildup = ' + str(np.round(Buildup_best,2)) 
worst_Bup_str = 'Buildup = ' + str(Buildup_worst)

if place_text:
    plt.text(text_index, -45, best_Bup_str, fontsize = 12)
    plt.text(text_index, -65, worst_Bup_str, fontsize = 12)
    plt.arrow(text_index+3, -47, 0, -10, width=0.05, shape="full", head_length = 1, head_width = 0.5, color = 'k',zorder=40)
    
#     plt.text(text_index2, -21, best_Bup_str, fontsize = 12)
#     plt.text(text_index2, -35, worst_Bup_str, fontsize = 12)
    plt.arrow(text_index2, -25, 0, -9, width=0.05, shape="full", head_length = 1, head_width = 0.5, color = 'k',zorder=40)

plt.xlim(0,25)

# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
# plt.legend(handlelength=3, bbox_to_anchor=(1.57, .8),loc='right')
plt.legend(handlelength=3, loc='best')
plt.subplot(1, 2, 2)
color_adj_start1 = 1
color_adj_start2 = 1
color_increment1 = -.4
color_increment2 = -.2
# Noise Figure for loop with RIN sweep
for i in range(len(Buildup_sweep)):
    if i == 0: 
        plt.plot(Pin,NF_EM[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label=label_EM)
        plt.plot(Pin,NF_DM[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label=label_DM)
    else: 
        plt.plot(Pin,NF_EM[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label='_nolegend_')
        plt.plot(Pin,NF_DM[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label='_nolegend_')


plt.ylabel('Noise Figure (dB)', fontsize=fs)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)
# plt.title('RF Link Loss of Rx path for Canceller Architectures ', fontsize=15)

text_index = 0.5
text_index2 = 1
best_Bup_str = 'Buildup = ' + str(np.round(Buildup_best,2)) 
worst_Bup_str = 'Buildup = ' + str(Buildup_worst)

if place_text:
    plt.text(text_index, 57, best_Bup_str, fontsize = 12)
    plt.text(text_index, 41, worst_Bup_str, fontsize = 12)
    plt.arrow(text_index+3, 55, 0, -10, width=0.05, shape="full", head_length = 1, head_width = 0.5, color = 'k',zorder=40)
    
#     plt.text(text_index2, 30, best_Bup_str, fontsize = 12)
#     plt.text(text_index2, 20, worst_Bup_str, fontsize = 12)
    plt.arrow(text_index2,35, 0, -9, width=0.05, shape="full", head_length = 1, head_width = 0.5, color = 'k',zorder=40)


plt.xlim(0,25)

# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
# plt.legend(handlelength=3, bbox_to_anchor=(1.57, .8),loc='right')
plt.legend(handlelength=3, loc='best')
plt.tight_layout()

if isSaving:
    outputFile = 'TFig2324.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res)
plt.show()

In [ ]:

plt.figure(figsize=(7,5))
color_adj_start1 = 1
color_adj_start2 = 1
color_increment1 = -.4
color_increment2 = -.2
# Noise Figure for loop with RIN sweep
for i in range(len(Buildup_sweep)):
    if i == 0: 
        plt.plot(Pin,SFDR_3[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label=label_EM)
        plt.plot(Pin,SFDR_3DML[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label=label_DM)
    else: 
        plt.plot(Pin,SFDR_3[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label='_nolegend_')
        plt.plot(Pin,SFDR_3DML[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label='_nolegend_')


# plt.vlines(10,125,110, color ='k', linestyles='dashed',linewidth=line_w+1)
# plt.text(0.5, 104, 'Small Signal Modulation\n Region of Direct Mod.', fontsize = 12)
# plt.arrow(9.5, 112, -3, 0, width=0.5, shape="full", head_length = 0.3, head_width = 2, color = 'k')

text_index = 1
text_index2 = 3
best_Bup_str = 'Buildup = ' + str(np.round(Buildup_best,2)) 
worst_Bup_str = 'Buildup = ' + str(Buildup_worst)

if place_text:
    plt.text(text_index, 86.5, best_Bup_str, fontsize = 12)
    plt.text(text_index, 80, worst_Bup_str, fontsize = 12)
    plt.arrow(text_index+2, 85.5, 0, -4, width=0.05, shape="full", head_length = 0.5, head_width = 0.5, color = 'k',zorder=40)
    
#     plt.text(text_index2, 30, best_Bup_str, fontsize = 12)
#     plt.text(text_index2, 20, worst_Bup_str, fontsize = 12)
    plt.arrow(text_index2,94.5, 0, -6, width=0.05, shape="full", head_length = 0.5, head_width = 0.5, color = 'k',zorder=40)


plt.ylabel('Spurious-Free Dynmaic Range (dB/Hz^2/3)', fontsize=fs-2)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)
plt.xlim(0,25)
# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
# plt.legend(handlelength=3, bbox_to_anchor=(1.57, .8),loc='right')
plt.legend(handlelength=3, loc='best')
if isSaving:
    outputFile = 'TFig25_SFDR_EMDM_wNonlinearloss_buildup_sweep.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res)
plt.show()
# 

In [ ]:
# Sweep Vpi and S_LD output ll, nf, sfdr

In [ ]:
Vpi_sweep = np.linspace(Vpi_best,Vpi_worst,5) #dBm

G_EM = np.zeros((len(Vpi_sweep),len(Pin)))
G_EM_LNA = np.zeros((len(Vpi_sweep),len(Pin)))
G_EM_lin = np.zeros((len(Vpi_sweep),len(Pin)))
NF_EM = np.zeros((len(Vpi_sweep),len(Pin)))
NF_EM_LNA = np.zeros((len(Vpi_sweep),len(Pin)))
NF_EM_lin = np.zeros((len(Vpi_sweep),len(Pin)))
NF_EM_lin_LNA = np.zeros((len(Vpi_sweep),len(Pin)))
SFDR_3 = np.zeros((len(Vpi_sweep),len(Pin)))
SFDR_EM_LNA = np.zeros((len(Vpi_sweep),len(Pin)))

ODEres = 100 

##############
#Nonlinear WG ODE off-chip mod

Pin_lin_wg = 10**((Pin+ECoupler)/10)/1000

#############
#Waveguide Length
WgL = 0.2 # cm
# distance points points
Wgz = np.linspace(0,WgL,ODEres) #Waveguide distance

#Nonlinear loss
Gopt_nonlin = np.linspace(0,0,len(Pin))
i=0
for y0 in Pin_lin_wg:
    # solve ODE
    Pin_z = odeint(model,y0,Wgz)
    Gopt_nonlin[i] = 10*np.log10(Pin_z[-1]/Pin_z[0])
    i+=1


####### Nonlinear MRR Loss 
Pin_lin_wg_mrr = (10**((Pin+ECoupler+Tmod)/10)/1000)*RefBuildup

#############
#Waveguide Length
WgL = c_bssmrr*1e-4 # cm
# distance points points
Wgz = np.linspace(0,WgL,ODEres) #Waveguide distance

#Nonlinear loss
Gopt_nonlin_mrr = np.linspace(0,0,len(Pin))
i=0
for y0 in Pin_lin_wg_mrr:
    # solve ODE
    Pin_z = odeint(model,y0,Wgz)
    Gopt_nonlin_mrr[i] = 10*np.log10(Pin_z[-1]/Pin_z[0])
    i+=1
    

Gopt = -2+Gopt_nonlin+Gopt_nonlin_mrr #dB

#convert to linear
Gopt_lin = 10**(Gopt/10)
Tmod_lin = 10**(Tmod/10)

for i in range(len(Vpi_sweep)):
    # Link Loss
    G_EM_lin[i,:] = (1/4)*((np.pi*Pin_lin*Tmod_lin*RL)/(2*Vpi_sweep[i]))**2*Gopt_lin**2*Rpd**2 ##lin
    G_EM[i,:] = 10*np.log10(G_EM_lin[i,:]) ##dB
    G_EM_LNA[i,:] = G_EM[i,:] + LNA_G

    # Noise Figure
    NF_EM_lin[i,:] = 1 + \
        (16*Vpi_sweep[i]**2)/(np.pi**2 * Pin_lin**2 * Gopt_lin **2 * Rpd **2 * RL **2) + \
        (16*q*Vpi_sweep[i]**2)/(np.pi**2 * Pin_lin * Gopt_lin * Rpd * RL*k*T) + \
        (4*Vpi_sweep[i]**2*RIN_lin)/(np.pi**2*RL*k*T)
    NF_EM[i,:] = 10*np.log10(NF_EM_lin[i,:]) ##dB

    #LNA Friis
    NF_EM_lin_LNA[i,:] = LNA_NF_lin+(NF_EM_lin[i,:]-1)/LNA_G_lin
    NF_EM_LNA[i,:] = 10*np.log10(NF_EM_lin_LNA[i,:]) ##dB

    # SFDR
    SFDR_3[i,:] = (2/3)*((4*Vpi_sweep[i]**2)/(np.pi**2*RL) - NF_EM[i,:] + 174 ) 
    SFDR_EM_LNA[i,:] = SFDR_3[i,:] + (NF_EM[i,:]-NF_EM_LNA[i,:])-LNA_G


In [ ]:
Sld_sweep = np.linspace(Sld_best,Sld_worst,5) #dBm

G_DM = np.zeros((len(Sld_sweep),len(Pin)))
G_DM_LNA = np.zeros((len(Sld_sweep),len(Pin)))
G_DM_lin = np.zeros((len(Sld_sweep),len(Pin)))
NF_DM = np.zeros((len(Sld_sweep),len(Pin)))
F_DM_lin_LNA = np.zeros((len(Sld_sweep),len(Pin)))
NF_DM_LNA = np.zeros((len(Sld_sweep),len(Pin)))
F_DM_lin = np.zeros((len(Sld_sweep),len(Pin)))
DML_IIP3 = np.zeros((len(Sld_sweep),len(Pin)))
SFDR_3DML = np.zeros((len(Sld_sweep),len(Pin)))
SFDR_DML_LNA = np.zeros((len(Sld_sweep),len(Pin)))

#Typical from Charles Cox / ILX /
# Consider APIC Linear DFB Laser
# http://www.cybel-llc.com/wp-content/uploads/2018/07/DM-DFB-Laser_030818a-v1.pdf
WgL = 0.2 # cm
Wgz = np.linspace(0,WgL,21) #Waveguide distance

#Nonlinear loss
Gopt_nonlin = np.linspace(0,0,len(Pin))
Pin_lin_wg = (10**((Pin+ECoupler)/10)/1000)*RefBuildup
i=0
for y0 in Pin_lin_wg:
    # solve ODE
    Pin_z = odeint(model,y0,Wgz)
    Gopt_nonlin[i] = 10*np.log10(Pin_z[-1]/Pin_z[0])
    i+=1

####### Nonlinear MRR Loss 
Pin_lin_wg_mrr = (10**((Pin+ECoupler)/10)/1000)*RefBuildup

#############
#Waveguide Length
WgL = c_bssmrr*1e-4 # cm
# distance points points
Wgz = np.linspace(0,WgL,11) #Waveguide distance

#Nonlinear loss
Gopt_nonlin_mrr = np.linspace(0,0,len(Pin))
i=0
for y0 in Pin_lin_wg_mrr:
    # solve ODE
    Pin_z = odeint(model,y0,Wgz)
    Gopt_nonlin_mrr[i] = 10*np.log10(Pin_z[-1]/Pin_z[0])
    i+=1
    
    
Gopt = -2+Gopt_nonlin+Gopt_nonlin_mrr #dB

#convert to linear
Gopt_lin = 10**(Gopt/10)

for i in range(len(Sld_sweep)):
    # Link Loss
    G_DM_lin[i,:] = np.full((1, len(Pin)),(1/4)*(Sld_sweep[i]**2*Gopt_lin**2*Rpd**2))##lin
    G_DM[i,:]  = 10*np.log10(G_DM_lin[i,:] ) ##dB
    G_DM_LNA[i,:]  = G_DM[i,:]  + LNA_G


    #Noise Figure
    F_DM_lin[i,:]  = 1+(4/((Sld_sweep[i]*Gopt_lin*Rpd)**2)  + \
        (2*q*Pin_lin*RL)/(Sld_sweep[i]**2*Gopt_lin*Rpd*k*T)+ \
        (RIN_lin*Pin_lin**2*RL)/(Sld_sweep[i]**2*k*T))

    NF_DM[i,:]  = 10*np.log10(F_DM_lin[i,:] ) ##dB

    #LNA Friis
    F_DM_lin_LNA[i,:]  = LNA_NF_lin+(F_DM_lin[i,:] -1)/LNA_G_lin
    NF_DM_LNA[i,:]  = 10*np.log10(F_DM_lin_LNA[i,:] ) ##dB


    # SFDR for DML
    DML_IIP3[i,:]  = 10*np.log10((c*Pin_lin**2*RL)/(2*Sld_sweep[i]))
    SFDR_3DML[i,:]  = (2/3)*(DML_IIP3[i,:]  - NF_DM[i,:]  + 174 ) 
    SFDR_DML_LNA[i,:]  = SFDR_3DML[i,:]  + (NF_DM[i,:] -NF_DM_LNA[i,:])-LNA_G


In [ ]:
### Plot both sweeps of discrete link loss together
# Plot Link Loss verse Optical power / Sweep SLD
plt.figure(figsize=(7,5))
color_adj_start = 1
color_increment = -0.4
for i in range(len(Vpi_sweep)):
    if i == 0:
        plt.plot(Pin,G_EM[i,:], color = adjust_lightness(color_EM,color_adj_start-i*color_increment), linestyle = line_EM, linewidth=line_w,label=label_EM)
    else: 
        plt.plot(Pin,G_EM[i,:], color = adjust_lightness(color_EM,color_adj_start-i*color_increment), linestyle = line_EM, linewidth=line_w, label='_nolegend_')

i=0
color_adj_start = 1
color_increment = -0.2
for i in range(len(Sld_sweep)):
    if i == 0:
        plt.plot(Pin,G_DM[i,:], color = adjust_lightness(color_DM,color_adj_start-i*color_increment), linestyle = line_DM, linewidth=line_w,label=label_DM)
    else: 
        plt.plot(Pin,G_DM[i,:], color = adjust_lightness(color_DM,color_adj_start-i*color_increment), linestyle = line_DM, linewidth=line_w, label='_nolegend_')

        
plt.ylabel('Link Loss (dB)', fontsize=fs)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)

text_index = 1
text_index2 =21.5

if place_text:
    plt.text(text_index, -20, best_sld_str, fontsize = 11)
    plt.text(text_index, -46, worst_sld_str, fontsize = 11)
    plt.arrow(text_index+2, -22, 0, -20, width=0.05, shape="full", head_length = 1, head_width = 0.5, color = 'k',zorder=40)
    
    plt.text(text_index2, -24, best_vpi_str, fontsize = 11)
    plt.text(text_index2, -58, worst_vpi_str, fontsize = 11)
    plt.arrow(text_index2+1.5, -25, 0, -29, width=0.05, shape="full", head_length = 1, head_width = 0.5, color = 'k',zorder=40)


plt.xlim(0,25)
plt.ylim(-90,-10)
# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
# plt.legend(handlelength=3, bbox_to_anchor=(1.57, .8),loc='right')
plt.legend(handlelength=3, loc='lower right')
if isSaving:
    outputFile = 'TFig28_EMDM_LL_VpiSld_sweep.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res, bbox_inches='tight')
plt.show()


In [ ]:
### Plot both sweeps of discrete link loss together
# Plot Link Loss verse Optical power / Sweep SLD
plt.figure(figsize=(7,5))
color_adj_start = 1
color_increment = -0.4
for i in range(len(Vpi_sweep)):
    if i == 0:
        plt.plot(Pin,NF_EM[i,:], color = adjust_lightness(color_EM,color_adj_start-i*color_increment), linestyle = line_EM, linewidth=line_w,label=label_EM)
    else: 
        plt.plot(Pin,NF_EM[i,:], color = adjust_lightness(color_EM,color_adj_start-i*color_increment), linestyle = line_EM, linewidth=line_w, label='_nolegend_')

i=0
color_adj_start = 1
color_increment = -0.2
for i in range(len(Sld_sweep)):
    if i == 0:
        plt.plot(Pin,NF_DM[i,:], color = adjust_lightness(color_DM,color_adj_start-i*color_increment), linestyle = line_DM, linewidth=line_w,label=label_DM)
    else: 
        plt.plot(Pin,NF_DM[i,:], color = adjust_lightness(color_DM,color_adj_start-i*color_increment), linestyle = line_DM, linewidth=line_w, label='_nolegend_')

        
plt.ylabel('Noise Figure (dB)', fontsize=fs)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)


text_index = 0.1
text_index2 =21.5

if place_text:
    plt.text(text_index, 21, best_sld_str, fontsize = 11)
    plt.text(text_index, 44.1, worst_sld_str, fontsize = 11)
    plt.arrow(text_index+1.8, 42, 0, -18.5, width=0.05, shape="full", head_length = 1, head_width = 0.5, color = 'k',zorder=40)
    
    plt.text(text_index2, 25, best_vpi_str, fontsize = 11)
    plt.text(text_index2, 55, worst_vpi_str, fontsize = 11)
    plt.arrow(text_index2+1.5, 27, 0, 26, width=0.05, shape="full", head_length = 1, head_width = 0.5, color = 'k',zorder=40)



plt.xlim(0,25)
plt.ylim(20,70)
# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
# plt.legend(handlelength=3, bbox_to_anchor=(1.57, .8),loc='right')
plt.legend(handlelength=3, loc='upper left')
if isSaving:
    outputFile = 'TFig29_EMDM_NF_VpiSld_sweep.pdf'
plt.savefig(outputFile, format='pdf', dpi=dpi_res, bbox_inches='tight')
plt.show()


In [ ]:
fig = plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
color_adj_start = 1
color_increment = -0.4
for i in range(len(Vpi_sweep)):
    if i == 0:
        plt.plot(Pin,G_EM[i,:], color = adjust_lightness(color_EM,color_adj_start-i*color_increment), linestyle = line_EM, linewidth=line_w,label=label_EM)
    else: 
        plt.plot(Pin,G_EM[i,:], color = adjust_lightness(color_EM,color_adj_start-i*color_increment), linestyle = line_EM, linewidth=line_w, label='_nolegend_')

i=0
color_adj_start = 1
color_increment = -0.2
for i in range(len(Sld_sweep)):
    if i == 0:
        plt.plot(Pin,G_DM[i,:], color = adjust_lightness(color_DM,color_adj_start-i*color_increment), linestyle = line_DM, linewidth=line_w,label=label_DM)
    else: 
        plt.plot(Pin,G_DM[i,:], color = adjust_lightness(color_DM,color_adj_start-i*color_increment), linestyle = line_DM, linewidth=line_w, label='_nolegend_')

        
plt.ylabel('Link Loss (dB)', fontsize=fs)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)

text_index = 1
text_index2 =21.5

if place_text:
    plt.text(text_index, -20, best_sld_str, fontsize = 11)
    plt.text(text_index, -46, worst_sld_str, fontsize = 11)
    plt.arrow(text_index+2, -22, 0, -20, width=0.05, shape="full", head_length = 1, head_width = 0.5, color = 'k',zorder=40)
    
    plt.text(text_index2, -24, best_vpi_str, fontsize = 11)
    plt.text(text_index2, -58, worst_vpi_str, fontsize = 11)
    plt.arrow(text_index2+1.5, -25, 0, -29, width=0.05, shape="full", head_length = 1, head_width = 0.5, color = 'k',zorder=40)


plt.xlim(0,25)
plt.ylim(-90,-10)
# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
# plt.legend(handlelength=3, bbox_to_anchor=(1.57, .8),loc='right')
plt.legend(handlelength=3, loc='lower right')

plt.subplot(1, 2, 2)
color_adj_start = 1
color_increment = -0.4
for i in range(len(Vpi_sweep)):
    if i == 0:
        plt.plot(Pin,NF_EM[i,:], color = adjust_lightness(color_EM,color_adj_start-i*color_increment), linestyle = line_EM, linewidth=line_w,label=label_EM)
    else: 
        plt.plot(Pin,NF_EM[i,:], color = adjust_lightness(color_EM,color_adj_start-i*color_increment), linestyle = line_EM, linewidth=line_w, label='_nolegend_')

i=0
color_adj_start = 1
color_increment = -0.2
for i in range(len(Sld_sweep)):
    if i == 0:
        plt.plot(Pin,NF_DM[i,:], color = adjust_lightness(color_DM,color_adj_start-i*color_increment), linestyle = line_DM, linewidth=line_w,label=label_DM)
    else: 
        plt.plot(Pin,NF_DM[i,:], color = adjust_lightness(color_DM,color_adj_start-i*color_increment), linestyle = line_DM, linewidth=line_w, label='_nolegend_')

        
plt.ylabel('Noise Figure (dB)', fontsize=fs)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)


text_index = 0.1
text_index2 =21.5

if place_text:
    plt.text(text_index, 21, best_sld_str, fontsize = 11)
    plt.text(text_index, 44.1, worst_sld_str, fontsize = 11)
    plt.arrow(text_index+1.8, 42, 0, -18.5, width=0.05, shape="full", head_length = 1, head_width = 0.5, color = 'k',zorder=40)
    
    plt.text(text_index2, 25, best_vpi_str, fontsize = 11)
    plt.text(text_index2, 55, worst_vpi_str, fontsize = 11)
    plt.arrow(text_index2+1.5, 27, 0, 26, width=0.05, shape="full", head_length = 1, head_width = 0.5, color = 'k',zorder=40)



plt.xlim(0,25)
plt.ylim(20,70)
# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
# plt.legend(handlelength=3, bbox_to_anchor=(1.57, .8),loc='right')
plt.legend(handlelength=3, loc='upper left')
if isSaving:
    outputFile = 'TFig2829.pdf'
plt.savefig(outputFile, format='pdf', dpi=dpi_res, bbox_inches='tight')
plt.show()



In [ ]:
### Plot both sweeps of discrete link loss together
# Plot Link Loss verse Optical power / Sweep SLD
plt.figure(figsize=(7,5))
color_adj_start = 1
color_increment = -0.4
for i in range(len(Vpi_sweep)):
    if i == 0:
        plt.plot(Pin,SFDR_3[i,:], color = adjust_lightness(color_EM,color_adj_start-i*color_increment), linestyle = line_EM, linewidth=line_w,label=label_EM)
    else: 
        plt.plot(Pin,SFDR_3[i,:], color = adjust_lightness(color_EM,color_adj_start-i*color_increment), linestyle = line_EM, linewidth=line_w, label='_nolegend_')

i=0
color_adj_start = 1
color_increment = -0.2
for i in range(len(Sld_sweep)):
    if i == 0:
        plt.plot(Pin,SFDR_3DML[i,:], color = adjust_lightness(color_DM,color_adj_start-i*color_increment), linestyle = line_DM, linewidth=line_w,label=label_DM)
    else: 
        plt.plot(Pin,SFDR_3DML[i,:], color = adjust_lightness(color_DM,color_adj_start-i*color_increment), linestyle = line_DM, linewidth=line_w, label='_nolegend_')

text_index = 1.5
text_index2 = 15
if place_text:
    plt.text(text_index, 96, best_sld_str, fontsize = 12)
    plt.text(text_index, 87, worst_sld_str, fontsize = 12)
    plt.arrow(text_index+1.8,95.5, 0, -7, width=0.05, shape="full", head_length = 0.5, head_width = 0.5, color = 'k',zorder=40)
   
    plt.text(text_index2-2, 82, worst_vpi_str, fontsize = 12)
    plt.text(text_index2-2, 100.5, best_vpi_str, fontsize = 12)
    plt.arrow(text_index2,100, 0, -16.5, width=0.05, shape="full", head_length = 0.5, head_width = 0.5, color = 'k',zorder=40)
    
    
plt.ylabel('Spurious-Free Dynamic Range (dB/Hz^2/3)', fontsize=fs-2)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)



plt.xlim(0,25)
plt.ylim(75,110)
# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
# plt.legend(handlelength=3, bbox_to_anchor=(1.57, .8),loc='right')
plt.legend(handlelength=3, loc='lower right')
if isSaving:
    outputFile = 'TFig30_EMDM_SFDR_VpiSld_sweep.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res, bbox_inches='tight')
plt.show()


In [ ]:
### Plot both sweeps of discrete link loss together
# Plot Link Loss verse Optical power / Sweep SLD
plt.figure(figsize=(7,5))
color_adj_start = 1
color_increment = -0.4
for i in range(len(Vpi_sweep)):
    if i == 0:
        plt.plot(Pin,SFDR_EM_LNA[i,:], color = adjust_lightness(color_EM,color_adj_start-i*color_increment), linestyle = line_EM, linewidth=line_w,label=label_EM+'w/ LNA')
    else: 
        plt.plot(Pin,SFDR_EM_LNA[i,:], color = adjust_lightness(color_EM,color_adj_start-i*color_increment), linestyle = line_EM, linewidth=line_w, label='_nolegend_')

i=0
color_adj_start = 1
color_increment = -0.2
for i in range(len(Sld_sweep)):
    if i == 0:
        plt.plot(Pin,SFDR_DML_LNA[i,:], color = adjust_lightness(color_DM,color_adj_start-i*color_increment), linestyle = line_DM, linewidth=line_w,label=label_DM+'w/ LNA')
    else: 
        plt.plot(Pin,SFDR_DML_LNA[i,:], color = adjust_lightness(color_DM,color_adj_start-i*color_increment), linestyle = line_DM, linewidth=line_w, label='_nolegend_')

        
plt.ylabel('Spurious-Free Dynamic Range (dB/Hz^2/3)', fontsize=fs-2)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)

text_index = 3
text_index2 = 15
if place_text:
    plt.text(text_index-1.5, 95, best_sld_str, fontsize = 12)
    plt.text(text_index-1.5, 86.5, worst_sld_str, fontsize = 12)
    plt.arrow(text_index, 94, 0, -6, width=0.05, shape="full", head_length = 0.5, head_width = 0.5, color = 'k',zorder=40)

    plt.text(text_index2-3, 98.5, best_vpi_str, fontsize = 12)
    plt.text(text_index2-3, 80, worst_vpi_str, fontsize = 12)
    plt.arrow(text_index2-1,81, 0, 16, width=0.05, shape="full", head_length = 0.5, head_width = 0.5, color = 'k',zorder=40)
    
plt.xlim(0,25)
plt.ylim(75,105)
# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
# plt.legend(handlelength=3, bbox_to_anchor=(1.57, .8),loc='right')
plt.legend(handlelength=3, loc='lower right')
if isSaving:
    outputFile = 'TFig31_EMDM_SFDR_VpiSld_sweep_wLNA.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res, bbox_inches='tight')
plt.show()


In [ ]:
fig = plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
color_adj_start = 1
color_increment = -0.4
for i in range(len(Vpi_sweep)):
    if i == 0:
        plt.plot(Pin,SFDR_3[i,:], color = adjust_lightness(color_EM,color_adj_start-i*color_increment), linestyle = line_EM, linewidth=line_w,label=label_EM)
    else: 
        plt.plot(Pin,SFDR_3[i,:], color = adjust_lightness(color_EM,color_adj_start-i*color_increment), linestyle = line_EM, linewidth=line_w, label='_nolegend_')

i=0
color_adj_start = 1
color_increment = -0.2
for i in range(len(Sld_sweep)):
    if i == 0:
        plt.plot(Pin,SFDR_3DML[i,:], color = adjust_lightness(color_DM,color_adj_start-i*color_increment), linestyle = line_DM, linewidth=line_w,label=label_DM)
    else: 
        plt.plot(Pin,SFDR_3DML[i,:], color = adjust_lightness(color_DM,color_adj_start-i*color_increment), linestyle = line_DM, linewidth=line_w, label='_nolegend_')

text_index = 1.5
text_index2 = 15
if place_text:
    plt.text(text_index, 96, best_sld_str, fontsize = 12)
    plt.text(text_index, 87, worst_sld_str, fontsize = 12)
    plt.arrow(text_index+1.8,95.5, 0, -7, width=0.05, shape="full", head_length = 0.5, head_width = 0.5, color = 'k',zorder=40)
   
    plt.text(text_index2-2, 82, worst_vpi_str, fontsize = 12)
    plt.text(text_index2-2, 100.5, best_vpi_str, fontsize = 12)
    plt.arrow(text_index2,100, 0, -16.5, width=0.05, shape="full", head_length = 0.5, head_width = 0.5, color = 'k',zorder=40)
    
plt.ylabel('Spurious-Free Dynamic Range (dB/Hz^2/3)', fontsize=fs-2)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)



plt.xlim(0,25)
plt.ylim(75,110)
# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
# plt.legend(handlelength=3, bbox_to_anchor=(1.57, .8),loc='right')
plt.legend(handlelength=3, loc='lower right')

plt.subplot(1, 2, 2)
color_adj_start = 1
color_increment = -0.4
for i in range(len(Vpi_sweep)):
    if i == 0:
        plt.plot(Pin,SFDR_EM_LNA[i,:], color = adjust_lightness(color_EM,color_adj_start-i*color_increment), linestyle = line_EM, linewidth=line_w,label=label_EM+'w/ LNA')
    else: 
        plt.plot(Pin,SFDR_EM_LNA[i,:], color = adjust_lightness(color_EM,color_adj_start-i*color_increment), linestyle = line_EM, linewidth=line_w, label='_nolegend_')

i=0
color_adj_start = 1
color_increment = -0.2
for i in range(len(Sld_sweep)):
    if i == 0:
        plt.plot(Pin,SFDR_DML_LNA[i,:], color = adjust_lightness(color_DM,color_adj_start-i*color_increment), linestyle = line_DM, linewidth=line_w,label=label_DM+'w/ LNA')
    else: 
        plt.plot(Pin,SFDR_DML_LNA[i,:], color = adjust_lightness(color_DM,color_adj_start-i*color_increment), linestyle = line_DM, linewidth=line_w, label='_nolegend_')

        
plt.ylabel('Spurious-Free Dynamic Range (dB/Hz^2/3)', fontsize=fs-2)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)

text_index = 3
text_index2 = 15
if place_text:
    plt.text(text_index-1.5, 95, best_sld_str, fontsize = 12)
    plt.text(text_index-1.5, 86.5, worst_sld_str, fontsize = 12)
    plt.arrow(text_index, 94, 0, -6, width=0.05, shape="full", head_length = 0.5, head_width = 0.5, color = 'k',zorder=40)

    plt.text(text_index2-3, 98.5, best_vpi_str, fontsize = 12)
    plt.text(text_index2-3, 80, worst_vpi_str, fontsize = 12)
    plt.arrow(text_index2-1,81, 0, 16, width=0.05, shape="full", head_length = 0.5, head_width = 0.5, color = 'k',zorder=40)
    
plt.xlim(0,25)
plt.ylim(75,110)
# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
# plt.legend(handlelength=3, bbox_to_anchor=(1.57, .8),loc='right')
plt.legend(handlelength=3, loc='lower right')
if isSaving:
    outputFile = 'TFig3031.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res, bbox_inches='tight')

In [ ]:
### Figure out SFDR in more detail, sweep LNA G and then LNA NF

In [ ]:
### SFDR, Sweep LNA Gain
# Standard Link
LNA_G_best = 30
LNA_G_worst = 15

LNA_G_sweep = np.linspace(LNA_G_worst,LNA_G_best,5)
LNA_G_lin_sweep = [10**(x/10) for x in LNA_G_sweep]

Gopt = np.zeros((1,len(Pin)))
Gopt_lin = np.zeros((1,len(Pin)))
G_DM = np.zeros((1,len(Pin)))
G_DM_LNA = np.zeros((len(LNA_G_sweep),len(Pin)))
G_DM_lin = np.zeros((1,len(Pin)))
NF_DM = np.zeros((1,len(Pin)))
NF_DM_LNA = np.zeros((len(LNA_G_sweep),len(Pin)))
NF_DM_lin_LNA = np.zeros((len(LNA_G_sweep),len(Pin)))
NF_DM_lin = np.zeros((1,len(Pin)))
SFDR_3DML = np.zeros((1,len(Pin)))
SFDR_3DML_lin = np.zeros((1,len(Pin)))
SFDR_DML_LNA = np.zeros((len(LNA_G_sweep),len(Pin)))

#Typical from Charles Cox / ILX /
# Consider APIC Linear DFB Laser
# http://www.cybel-llc.com/wp-content/uploads/2018/07/DM-DFB-Laser_030818a-v1.pdf
WgL = 0.2 # cm
Wgz = np.linspace(0,WgL,21) #Waveguide distance

#Nonlinear loss
Gopt_nonlin = np.linspace(0,0,len(Pin))
Pin_lin_wg = (10**((Pin+ECoupler)/10)/1000)*RefBuildup
i=0
for y0 in Pin_lin_wg:
    # solve ODE
    Pin_z = odeint(model,y0,Wgz)
    Gopt_nonlin[i] = 10*np.log10(Pin_z[-1]/Pin_z[0])
    i+=1

####### Nonlinear MRR Loss 
Pin_lin_wg_mrr = (10**((Pin+ECoupler)/10)/1000)*RefBuildup

#############
#Waveguide Length
WgL = c_bssmrr*1e-4 # cm
# distance points points
Wgz = np.linspace(0,WgL,11) #Waveguide distance

#Nonlinear loss
Gopt_nonlin_mrr = np.linspace(0,0,len(Pin))
i=0
for y0 in Pin_lin_wg_mrr:
    # solve ODE
    Pin_z = odeint(model,y0,Wgz)
    Gopt_nonlin_mrr[i] = 10*np.log10(Pin_z[-1]/Pin_z[0])
    i+=1
    
Gopt = -2+Gopt_nonlin+Gopt_nonlin_mrr #dB

#convert to linear
Gopt_lin = 10**(Gopt/10)

# Link Loss
G_DM_lin = np.full((1, len(Pin)),(1/4)*(S_LD**2*Gopt_lin**2*Rpd**2))##lin


#Noise Figure
F_DM_lin = 1+(4/((S_LD*Gopt_lin*Rpd)**2)  + \
    (2*q*Pin_lin*RL)/(S_LD**2*Gopt_lin*Rpd*k*T)+ \
    (RIN_lin*Pin_lin**2*RL)/(S_LD**2*k*T))
NF_DM = 10*np.log10(F_DM_lin) ##dB

G_DM = 10*np.log10(G_DM_lin) ##dB
# SFDR for DML
DML_IIP3 = 10*np.log10((c*Pin_lin**2*RL)/(2*S_LD))
SFDR_3DML = (2/3)*(DML_IIP3 - NF_DM + 174 ) 

for i in range(len(LNA_G_lin_sweep)):
    G_DM_LNA[i,:] = G_DM + LNA_G_sweep[i]
    #LNA Friis
    NF_DM_lin_LNA[i,:] = LNA_NF_lin+(F_DM_lin-1)/LNA_G_lin_sweep[i]
    NF_DM_LNA[i,:]  = 10*np.log10(NF_DM_lin_LNA[i,:]) ##dB
    SFDR_DML_LNA[i,:]  = SFDR_3DML + (NF_DM-NF_DM_LNA[i,:])-LNA_G_sweep[i]


In [ ]:
### SFDR, Sweep LNA Gain
# Standard Link
LNA_G_best = 30
LNA_G_worst = 15

LNA_G_sweep = np.linspace(LNA_G_worst,LNA_G_best,5)
LNA_G_lin_sweep = [10**(x/10) for x in LNA_G_sweep]


G_EM = np.zeros((len(Buildup_sweep),len(Pin)))

G_EM_lin = np.zeros((1,len(Pin)))
NF_EM = np.zeros((1,len(Pin)))
NF_EM_lin = np.zeros((1,len(Pin)))
SFDR_3 = np.zeros((1,len(Pin)))
SFDR_3_lin = np.zeros((1,len(Pin)))

NF_EM_LNA = np.zeros((len(LNA_G_sweep),len(Pin)))
G_EM_LNA = np.zeros((len(LNA_G_sweep),len(Pin)))
NF_EM_lin_LNA = np.zeros((len(LNA_G_sweep),len(Pin)))
SFDR_EM_LNA = np.zeros((len(LNA_G_sweep),len(Pin)))

ODEres = 100 

##############
#Nonlinear WG ODE off-chip mod

Pin_lin_wg = 10**((Pin+ECoupler)/10)/1000

#############
#Waveguide Length
WgL = 0.2 # cm
# distance points points
Wgz = np.linspace(0,WgL,ODEres) #Waveguide distance

#Nonlinear loss
Gopt_nonlin = np.linspace(0,0,len(Pin))
i=0
for y0 in Pin_lin_wg:
    # solve ODE
    Pin_z = odeint(model,y0,Wgz)
    Gopt_nonlin[i] = 10*np.log10(Pin_z[-1]/Pin_z[0])
    i+=1


# distance points points
WgL = c_bssmrr*1e-4 # cm
Wgz = np.linspace(0,WgL,31) #Waveguide distance


####### Nonlinear MRR Loss 
Pin_lin_wg_mrr = (10**((Pin+ECoupler)/10)/1000)*RefBuildup
#Nonlinear loss
Gopt_nonlin_mrr = np.linspace(0,0,len(Pin))
i=0
for y0 in Pin_lin_wg_mrr:
    # solve ODE
    Pin_z = odeint(model,y0,Wgz)
    Gopt_nonlin_mrr[i] = 10*np.log10(Pin_z[-1]/Pin_z[0])
    i+=1

Gopt = -2+Gopt_nonlin+Gopt_nonlin_mrr #dB

#convert to linear
Gopt_lin  = 10**(Gopt/10)


# Link Loss
G_EM_lin  = (1/4)*((np.pi*Pin_lin*Tmod_lin*RL)/(2*Vpi))**2*Gopt_lin**2*Rpd**2 ##lin
G_EM  = 10*np.log10(G_EM_lin) ##dB
G_EM_LNA  = G_EM + LNA_G

# Noise Figure
NF_EM_lin = 1 + \
    (16*Vpi**2)/(np.pi**2 * Pin_lin**2 * Gopt_lin**2 * Rpd **2 * RL **2) + \
    (16*q*Vpi**2)/(np.pi**2 * Pin_lin * Gopt_lin * Rpd * RL*k*T) + \
    (4*Vpi**2*RIN_lin)/(np.pi**2*RL*k*T)
NF_EM  = 10*np.log10(NF_EM_lin) ##dB
SFDR_3  = (2/3)*((4*Vpi**2)/(np.pi**2*RL) - NF_EM + 174 ) 
    
for z in range(len(LNA_G_sweep)):
    #LNA Friis
    NF_EM_lin_LNA[z,:]  = LNA_NF_lin+(NF_EM_lin-1)/LNA_G_lin_sweep[z]
    NF_EM_LNA[z,:]  = 10*np.log10(NF_EM_lin_LNA[z,:]) ##dB

    # SFDR

    SFDR_EM_LNA[z,:]  = SFDR_3  + (NF_EM - NF_EM_LNA[z,:] )-LNA_G_sweep[z]


In [ ]:
### EM and DM, Sweep LNA NF
plt.figure(figsize=(7,5))
color_adj_start1 = 1
color_adj_start2 = 1
color_increment1 = -.4
color_increment2 = -.2
# Noise Figure for loop with RIN sweep
for i in range(len(LNA_G_sweep)):
    if i == 0: 
        plt.plot(Pin,SFDR_EM_LNA[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label=label_EM+label_LNA)
        plt.plot(Pin,SFDR_DML_LNA[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label=label_DM+label_LNA)
    else: 
        plt.plot(Pin,SFDR_EM_LNA[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label='_nolegend_')
        plt.plot(Pin,SFDR_DML_LNA[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label='_nolegend_')

plt.plot(Pin,SFDR_3, color = 'k', linestyle = line_EM, linewidth=line_w-0.5,label=label_EM+'w/o LNA')
plt.plot(Pin,SFDR_3DML, color = 'k', linestyle = line_DM, linewidth=line_w-0.5,label=label_DM+'w/o LNA')


plt.ylabel('Spurious-Free Dynamic Range (dB/Hz^2/3)', fontsize=fs-2)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)
# plt.title('RF Link Loss of Rx path for Canceller Architectures ', fontsize=15)

text_index = 10
text_index2 = 16
if place_text:
    plt.text(text_index-1.5, 97, best_LNA_G_str, fontsize = 12)
    plt.text(text_index-1.5, 90, worst_LNA_G_str, fontsize = 12)
    plt.arrow(text_index, 96, 0, -3, width=0.05, shape="full", head_length = 0.5, head_width = 0.5, color = 'k',zorder=40)

#     plt.text(text_index2-2, 90, best_LNA_G_str, fontsize = 12)
#     plt.text(text_index2-2, 83, worst_LNA_G_str, fontsize = 12)
    plt.arrow(text_index2-1,89, 0, -3, width=0.05, shape="full", head_length = 0.5, head_width = 0.5, color = 'k',zorder=40)
    


plt.xlim(0,25)
plt.legend(handlelength=3, loc='lower right')
plt.ylim(70,115)
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
# Below 10mW requires further discussion on signal clipping due to nonlinear region Ith-IBias 
# plt.vlines(10,110,106, color ='k', linestyles='dashed',linewidth=line_w)
# plt.text(0.5, 104, 'Small Signal Modulation\n Region of Direct Mod.', fontsize = 12)
# plt.arrow(9.5, 96, -3, 0, width=0.3, shape="full", head_length = 0.3, head_width = 1, color = 'k')

plt.legend(handlelength=3, loc='best')
if isSaving:
    outputFile = 'TFig26_SFDR_EMDM_wNonlinearloss_LNA_G_sweep.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res)
plt.show()
# 

In [ ]:
### EM and DM, Sweep LNA NF
plt.figure(figsize=(7,5))
color_adj_start1 = 1
color_adj_start2 = 1
color_increment1 = -.6
color_increment2 = -.2
# Noise Figure for loop with RIN sweep
for i in range(len(LNA_G_sweep)):
    if i == 0: 
        plt.plot(Pin,SFDR_EM_LNA[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label=label_EM+label_LNA)
        # plt.plot(Pin,SFDR_DML_LNA[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label=label_DM+label_LNA)
    else: 
        plt.plot(Pin,SFDR_EM_LNA[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w-1,label='_nolegend_')
        # plt.plot(Pin,SFDR_DML_LNA[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label='_nolegend_')

plt.plot(Pin,SFDR_3, color = 'k', linestyle = line_EM, linewidth=line_w+2,label=label_EM+'w/o LNA')
# plt.plot(Pin,SFDR_3DML, color = 'k', linestyle = line_DM, linewidth=line_w-0.5,label=label_DM)


plt.ylabel('Spurious-Free Dynamic Range (dB/Hz^2/3)', fontsize=fs-2)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)
# plt.title('RF Link Loss of Rx path for Canceller Architectures ', fontsize=15)

# text_index = 10
text_index2 = 14
if place_text:
#     plt.text(text_index-1.5, 101, best_sld_str, fontsize = 12)
#     plt.text(text_index-1.5, 92, worst_sld_str, fontsize = 12)
#     plt.arrow(text_index, 100.5, 0, -5.5, width=0.05, shape="full", head_length = 0.5, head_width = 0.5, color = 'k',zorder=40)

    plt.text(text_index2-1, 88.3, best_LNA_G_str, fontsize = 12)
    plt.text(text_index2-1, 87.2, worst_LNA_G_str, fontsize = 12)
    plt.arrow(text_index2,88.25, 0, -.8, width=0.05, shape="full", head_length = 0.1, head_width = 0.3, color = 'k',zorder=40)
    


plt.xlim(8,20)
plt.ylim(86,88.5)
# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
# plt.legend(handlelength=3, bbox_to_anchor=(1.57, .8),loc='right')
plt.legend(handlelength=3, loc='best')
plt.legend(handlelength=3, loc='best')
if isSaving:
    outputFile = 'TFig26_SFDR_EM_wNonlinearloss_LNA_G_sweep_zoomedin.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res)
plt.show()
# 

In [ ]:
fig = plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
color_adj_start1 = 1
color_adj_start2 = 1
color_increment1 = -.4
color_increment2 = -.2
# Noise Figure for loop with RIN sweep
for i in range(len(LNA_G_sweep)):
    if i == 0: 
        plt.plot(Pin,SFDR_EM_LNA[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label=label_EM+label_LNA)
        plt.plot(Pin,SFDR_DML_LNA[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label=label_DM+label_LNA)
    else: 
        plt.plot(Pin,SFDR_EM_LNA[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label='_nolegend_')
        plt.plot(Pin,SFDR_DML_LNA[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label='_nolegend_')

plt.plot(Pin,SFDR_3, color = 'k', linestyle = line_EM, linewidth=line_w-0.5,label=label_EM+'w/o LNA')
plt.plot(Pin,SFDR_3DML, color = 'k', linestyle = line_DM, linewidth=line_w-0.5,label=label_DM+'w/o LNA')


plt.ylabel('Spurious-Free Dynamic Range (dB/Hz^2/3)', fontsize=fs-2)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)
# plt.title('RF Link Loss of Rx path for Canceller Architectures ', fontsize=15)

text_index = 10
text_index2 = 16
if place_text:
    plt.text(text_index-1.5, 97, best_LNA_G_str, fontsize = 12)
    plt.text(text_index-1.5, 90, worst_LNA_G_str, fontsize = 12)
    plt.arrow(text_index, 96, 0, -3, width=0.05, shape="full", head_length = 0.5, head_width = 0.5, color = 'k',zorder=40)

#     plt.text(text_index2-2, 90, best_LNA_G_str, fontsize = 12)
#     plt.text(text_index2-2, 83, worst_LNA_G_str, fontsize = 12)
    plt.arrow(text_index2-1,89, 0, -3, width=0.05, shape="full", head_length = 0.5, head_width = 0.5, color = 'k',zorder=40)
    
 


plt.xlim(0,25)
plt.legend(handlelength=3, loc='lower right')
plt.ylim(80,105)
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
# Below 10mW requires further discussion on signal clipping due to nonlinear region Ith-IBias 
# plt.vlines(10,110,106, color ='k', linestyles='dashed',linewidth=line_w)
# plt.text(0.5, 104, 'Small Signal Modulation\n Region of Direct Mod.', fontsize = 12)
# plt.arrow(9.5, 96, -3, 0, width=0.3, shape="full", head_length = 0.3, head_width = 1, color = 'k')

plt.legend(handlelength=3, loc='best')

plt.subplot(1, 2, 2)
color_adj_start1 = 1
color_adj_start2 = 1
color_increment1 = -.6
color_increment2 = -.2
# Noise Figure for loop with RIN sweep
for i in range(len(LNA_G_sweep)):
    if i == 0: 
        plt.plot(Pin,SFDR_EM_LNA[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label=label_EM+label_LNA)
        # plt.plot(Pin,SFDR_DML_LNA[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label=label_DM+label_LNA)
    else: 
        plt.plot(Pin,SFDR_EM_LNA[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w-1,label='_nolegend_')
        # plt.plot(Pin,SFDR_DML_LNA[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label='_nolegend_')

plt.plot(Pin,SFDR_3, color = 'k', linestyle = line_EM, linewidth=line_w+2,label=label_EM+'w/o LNA')
# plt.plot(Pin,SFDR_3DML, color = 'k', linestyle = line_DM, linewidth=line_w-0.5,label=label_DM)


plt.ylabel('Spurious-Free Dynamic Range (dB/Hz^2/3)', fontsize=fs-2)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)
# plt.title('RF Link Loss of Rx path for Canceller Architectures ', fontsize=15)

# text_index = 10
text_index2 = 14
if place_text:
#     plt.text(text_index-1.5, 101, best_sld_str, fontsize = 12)
#     plt.text(text_index-1.5, 92, worst_sld_str, fontsize = 12)
#     plt.arrow(text_index, 100.5, 0, -5.5, width=0.05, shape="full", head_length = 0.5, head_width = 0.5, color = 'k',zorder=40)

    plt.text(text_index2-1, 88.3, best_LNA_G_str, fontsize = 12)
    plt.text(text_index2-1, 87.2, worst_LNA_G_str, fontsize = 12)
    plt.arrow(text_index2,88.25, 0, -.8, width=0.05, shape="full", head_length = 0.1, head_width = 0.3, color = 'k',zorder=40)
    


plt.xlim(8,20)
plt.ylim(86,88.5)
# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
# plt.legend(handlelength=3, bbox_to_anchor=(1.57, .8),loc='right')
plt.legend(handlelength=3, loc='best')
plt.legend(handlelength=3, loc='best')
if isSaving:
    outputFile = 'TFig2626b.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res,bbox_inches='tight')

In [ ]:
### SFDR, Sweep LNA NF
# Standard Link
LNA_NF_best  = 3
LNA_NF_worst = 10

LNA_NF_sweep = np.linspace(LNA_NF_worst,LNA_NF_best,5)
LNA_NF_lin_sweep = [10**(x/10) for x in LNA_NF_sweep]

Gopt = np.zeros((1,len(Pin)))
Gopt_lin = np.zeros((1,len(Pin)))
G_DM = np.zeros((1,len(Pin)))
G_DM_LNA = np.zeros((len(LNA_NF_sweep),len(Pin)))
G_DM_lin = np.zeros((1,len(Pin)))
NF_DM = np.zeros((1,len(Pin)))
NF_DM_LNA = np.zeros((len(LNA_NF_sweep),len(Pin)))
NF_DM_lin_LNA = np.zeros((len(LNA_NF_sweep),len(Pin)))
NF_DM_lin = np.zeros((1,len(Pin)))
SFDR_3DML = np.zeros((1,len(Pin)))
SFDR_3DML_lin = np.zeros((1,len(Pin)))
SFDR_DML_LNA = np.zeros((len(LNA_NF_sweep),len(Pin)))

#Typical from Charles Cox / ILX /
# Consider APIC Linear DFB Laser
# http://www.cybel-llc.com/wp-content/uploads/2018/07/DM-DFB-Laser_030818a-v1.pdf
WgL = 0.2 # cm
Wgz = np.linspace(0,WgL,21) #Waveguide distance

#Nonlinear loss
Gopt_nonlin = np.linspace(0,0,len(Pin))
Pin_lin_wg = (10**((Pin+ECoupler)/10)/1000)*RefBuildup
i=0
for y0 in Pin_lin_wg:
    # solve ODE
    Pin_z = odeint(model,y0,Wgz)
    Gopt_nonlin[i] = 10*np.log10(Pin_z[-1]/Pin_z[0])
    i+=1

####### Nonlinear MRR Loss 
Pin_lin_wg_mrr = (10**((Pin+ECoupler)/10)/1000)*RefBuildup

#############
#Waveguide Length
WgL = c_bssmrr*1e-4 # cm
# distance points points
Wgz = np.linspace(0,WgL,11) #Waveguide distance

#Nonlinear loss
Gopt_nonlin_mrr = np.linspace(0,0,len(Pin))
i=0
for y0 in Pin_lin_wg_mrr:
    # solve ODE
    Pin_z = odeint(model,y0,Wgz)
    Gopt_nonlin_mrr[i] = 10*np.log10(Pin_z[-1]/Pin_z[0])
    i+=1
    
Gopt = -2+Gopt_nonlin+Gopt_nonlin_mrr #dB

#convert to linear
Gopt_lin = 10**(Gopt/10)

# Link Loss
G_DM_lin = np.full((1, len(Pin)),(1/4)*(S_LD**2*Gopt_lin**2*Rpd**2))##lin


#Noise Figure
F_DM_lin = 1+(4/((S_LD*Gopt_lin*Rpd)**2)  + \
    (2*q*Pin_lin*RL)/(S_LD**2*Gopt_lin*Rpd*k*T)+ \
    (RIN_lin*Pin_lin**2*RL)/(S_LD**2*k*T))
NF_DM = 10*np.log10(F_DM_lin) ##dB

G_DM = 10*np.log10(G_DM_lin) ##dB
# SFDR for DML
DML_IIP3 = 10*np.log10((c*Pin_lin**2*RL)/(2*S_LD))
SFDR_3DML = (2/3)*(DML_IIP3 - NF_DM + 174 ) 

for i in range(len(LNA_NF_sweep)):
    G_DM_LNA = G_DM + LNA_G
    #LNA Friis
    NF_DM_lin_LNA[i,:] = LNA_NF_lin_sweep[i]+(F_DM_lin-1)/LNA_G_lin
    NF_DM_LNA[i,:]  = 10*np.log10(NF_DM_lin_LNA[i,:]) ##dB
    SFDR_DML_LNA[i,:]  = SFDR_3DML + (NF_DM-NF_DM_LNA[i,:])-LNA_G


In [ ]:
### SFDR, Sweep LNA NF
# Standard Link
LNA_NF_best  = 3
LNA_NF_worst = 10

LNA_NF_sweep = np.linspace(LNA_NF_worst,LNA_NF_best,5)
LNA_NF_lin_sweep = [10**(x/10) for x in LNA_NF_sweep]

G_EM = np.zeros((len(Buildup_sweep),len(Pin)))

G_EM_lin = np.zeros((1,len(Pin)))
NF_EM = np.zeros((1,len(Pin)))
NF_EM_lin = np.zeros((1,len(Pin)))
SFDR_3 = np.zeros((1,len(Pin)))
SFDR_3_lin = np.zeros((1,len(Pin)))

NF_EM_LNA = np.zeros((len(LNA_NF_sweep),len(Pin)))
G_EM_LNA = np.zeros((len(LNA_NF_sweep),len(Pin)))
NF_EM_lin_LNA = np.zeros((len(LNA_NF_sweep),len(Pin)))
SFDR_EM_LNA = np.zeros((len(LNA_NF_sweep),len(Pin)))

ODEres = 100 

##############
#Nonlinear WG ODE off-chip mod

Pin_lin_wg = 10**((Pin+ECoupler)/10)/1000

#############
#Waveguide Length
WgL = 0.2 # cm
# distance points points
Wgz = np.linspace(0,WgL,ODEres) #Waveguide distance

#Nonlinear loss
Gopt_nonlin = np.linspace(0,0,len(Pin))
i=0
for y0 in Pin_lin_wg:
    # solve ODE
    Pin_z = odeint(model,y0,Wgz)
    Gopt_nonlin[i] = 10*np.log10(Pin_z[-1]/Pin_z[0])
    i+=1


# distance points points
WgL = c_bssmrr*1e-4 # cm
Wgz = np.linspace(0,WgL,31) #Waveguide distance


####### Nonlinear MRR Loss 
Pin_lin_wg_mrr = (10**((Pin+ECoupler)/10)/1000)*RefBuildup
#Nonlinear loss
Gopt_nonlin_mrr = np.linspace(0,0,len(Pin))
i=0
for y0 in Pin_lin_wg_mrr:
    # solve ODE
    Pin_z = odeint(model,y0,Wgz)
    Gopt_nonlin_mrr[i] = 10*np.log10(Pin_z[-1]/Pin_z[0])
    i+=1

Gopt = -2+Gopt_nonlin+Gopt_nonlin_mrr #dB

#convert to linear
Gopt_lin  = 10**(Gopt/10)


# Link Loss
G_EM_lin  = (1/4)*((np.pi*Pin_lin*Tmod_lin*RL)/(2*Vpi))**2*Gopt_lin**2*Rpd**2 ##lin
G_EM  = 10*np.log10(G_EM_lin) ##dB
G_EM_LNA  = G_EM + LNA_G

# Noise Figure
NF_EM_lin = 1 + \
    (16*Vpi**2)/(np.pi**2 * Pin_lin**2 * Gopt_lin**2 * Rpd **2 * RL **2) + \
    (16*q*Vpi**2)/(np.pi**2 * Pin_lin * Gopt_lin * Rpd * RL*k*T) + \
    (4*Vpi**2*RIN_lin)/(np.pi**2*RL*k*T)
NF_EM  = 10*np.log10(NF_EM_lin) ##dB
SFDR_3  = (2/3)*((4*Vpi**2)/(np.pi**2*RL) - NF_EM + 174 ) 
    
for z in range(len(LNA_G_sweep)):
    #LNA Friis
    NF_EM_lin_LNA[z,:]  = LNA_NF_lin_sweep[z]+(NF_EM_lin-1)/LNA_G_lin
    NF_EM_LNA[z,:]  = 10*np.log10(NF_EM_lin_LNA[z,:]) ##dB

    # SFDR

    SFDR_EM_LNA[z,:]  = SFDR_3  + (NF_EM - NF_EM_LNA[z,:] )-LNA_G


In [ ]:
### EM and DM, Sweep LNA NF
plt.figure(figsize=(7,5))
color_adj_start1 = 1
color_adj_start2 = 1
color_increment1 = -.4
color_increment2 = -.2
# Noise Figure for loop with RIN sweep
for i in range(len(LNA_NF_sweep)):
    if i == 0: 
        plt.plot(Pin,SFDR_EM_LNA[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label=label_EM+label_LNA)
        plt.plot(Pin,SFDR_DML_LNA[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label=label_DM+label_LNA)
    else: 
        plt.plot(Pin,SFDR_EM_LNA[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label='_nolegend_')
        plt.plot(Pin,SFDR_DML_LNA[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label='_nolegend_')

plt.plot(Pin,SFDR_3, color = 'k', linestyle = line_EM, linewidth=line_w-0.5,label=label_EM+'w/o LNA')
plt.plot(Pin,SFDR_3DML, color = 'k', linestyle = line_DM, linewidth=line_w-0.5,label=label_DM+'w/o LNA')

# plt.vlines(10,125,110, color ='k', linestyles='dashed',linewidth=line_w+1)
# plt.text(0.5, 104, 'Small Signal Modulation\n Region of Direct Mod.', fontsize = 12)
# plt.arrow(9.5, 112, -3, 0, width=0.5, shape="full", head_length = 0.3, head_width = 2, color = 'k')

plt.ylabel('Spurious-Free Dynamic Range (dB/Hz^2/3)', fontsize=fs-2)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)
# plt.title('RF Link Loss of Rx path for Canceller Architectures ', fontsize=15)
plt.xlim(0,25)
plt.ylim(80,110)

text_index = 10
text_index2 = 17
if place_text:
    plt.text(text_index-1.5, 97, best_LNA_NF_str, fontsize = 12)
    plt.text(text_index-1.5, 91, worst_LNA_NF_str, fontsize = 12)
    plt.arrow(text_index, 96, 0, -3, width=0.05, shape="full", head_length = 0.5, head_width = 0.5, color = 'k',zorder=40)

#     plt.text(text_index2-2, 90, best_LNA_NF_str, fontsize = 12)
#     plt.text(text_index2-2, 83, worst_LNA_NF_str, fontsize = 12)
    plt.arrow(text_index2-1,89, 0, -3, width=0.05, shape="full", head_length = 0.5, head_width = 0.5, color = 'k',zorder=40)
    

    
# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
# plt.legend(handlelength=3, bbox_to_anchor=(1.57, .8),loc='right')
plt.legend(handlelength=3, loc='upper right')
if isSaving:
    outputFile = 'TFig27_SFDR_EM_wNonlinearloss_LNA_NF.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res)
plt.show()
# 

In [ ]:
### EM and DM, Sweep LNA NF
plt.figure(figsize=(7,5))
color_adj_start1 = 1
color_adj_start2 = 1
color_increment1 = -.4
color_increment2 = -.2
# Noise Figure for loop with RIN sweep
for i in range(len(LNA_NF_sweep)):
    if i == 0: 
        plt.plot(Pin,SFDR_EM_LNA[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label=label_EM+label_LNA)
#         plt.plot(Pin,SFDR_DML_LNA[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label=label_DM+label_LNA)
    else: 
        plt.plot(Pin,SFDR_EM_LNA[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label='_nolegend_')
#         plt.plot(Pin,SFDR_DML_LNA[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label='_nolegend_')

plt.plot(Pin,SFDR_3, color = 'k', linestyle = line_EM, linewidth=line_w-0.5,label=label_EM+'w/o LNA')
# plt.plot(Pin,SFDR_3DML, color = 'k', linestyle = line_DM, linewidth=line_w-0.5,label=label_DM)

# plt.vlines(10,125,110, color ='k', linestyles='dashed',linewidth=line_w+1)
# plt.text(0.5, 104, 'Small Signal Modulation\n Region of Direct Mod.', fontsize = 12)
# plt.arrow(9.5, 112, -3, 0, width=0.5, shape="full", head_length = 0.3, head_width = 2, color = 'k')

# text_index = 10
text_index2 = 14
if place_text:
#     plt.text(text_index-1.5, 101, best_sld_str, fontsize = 12)
#     plt.text(text_index-1.5, 92, worst_sld_str, fontsize = 12)
#     plt.arrow(text_index, 100.5, 0, -5.5, width=0.05, shape="full", head_length = 0.5, head_width = 0.5, color = 'k',zorder=40)

    plt.text(text_index2-1, 88.3, best_LNA_NF_str, fontsize = 12)
    plt.text(text_index2-1, 87.2, worst_LNA_NF_str, fontsize = 12)
    plt.arrow(text_index2,88.25, 0, -.8, width=0.05, shape="full", head_length = 0.1, head_width = 0.3, color = 'k',zorder=40)
    
    
plt.ylabel('Spurious-Free Dynamic Range (dB/Hz^2/3)', fontsize=fs-2)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)
# plt.title('RF Link Loss of Rx path for Canceller Architectures ', fontsize=15)
plt.xlim(8,20)
plt.ylim(86,88.5)

# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
# plt.legend(handlelength=3, bbox_to_anchor=(1.57, .8),loc='right')
plt.legend(handlelength=3, loc='best')
if isSaving:
    outputFile = 'TFig27b_SFDR_EM_wNonlinearloss_LNA_NFzoomed.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res)
plt.show()
# 

In [ ]:
fig = plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
color_adj_start1 = 1
color_adj_start2 = 1
color_increment1 = -.4
color_increment2 = -.2
# Noise Figure for loop with RIN sweep
for i in range(len(LNA_NF_sweep)):
    if i == 0: 
        plt.plot(Pin,SFDR_EM_LNA[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label=label_EM+label_LNA)
        plt.plot(Pin,SFDR_DML_LNA[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label=label_DM+label_LNA)
    else: 
        plt.plot(Pin,SFDR_EM_LNA[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label='_nolegend_')
        plt.plot(Pin,SFDR_DML_LNA[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label='_nolegend_')

plt.plot(Pin,SFDR_3, color = 'k', linestyle = line_EM, linewidth=line_w-0.5,label=label_EM+'w/o LNA')
plt.plot(Pin,SFDR_3DML, color = 'k', linestyle = line_DM, linewidth=line_w-0.5,label=label_DM+'w/o LNA')

# plt.vlines(10,125,110, color ='k', linestyles='dashed',linewidth=line_w+1)
# plt.text(0.5, 104, 'Small Signal Modulation\n Region of Direct Mod.', fontsize = 12)
# plt.arrow(9.5, 112, -3, 0, width=0.5, shape="full", head_length = 0.3, head_width = 2, color = 'k')

plt.ylabel('Spurious-Free Dynamic Range (dB/Hz^2/3)', fontsize=fs-2)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)
# plt.title('RF Link Loss of Rx path for Canceller Architectures ', fontsize=15)
plt.xlim(0,25)
plt.ylim(80,110)

text_index = 10
text_index2 = 17
if place_text:
    plt.text(text_index-1.5, 97, best_LNA_NF_str, fontsize = 12)
    plt.text(text_index-1.5, 91, worst_LNA_NF_str, fontsize = 12)
    plt.arrow(text_index, 96, 0, -3, width=0.05, shape="full", head_length = 0.5, head_width = 0.5, color = 'k',zorder=40)

#     plt.text(text_index2-2, 90, best_LNA_NF_str, fontsize = 12)
#     plt.text(text_index2-2, 83, worst_LNA_NF_str, fontsize = 12)
    plt.arrow(text_index2-1,89, 0, -3, width=0.05, shape="full", head_length = 0.5, head_width = 0.5, color = 'k',zorder=40)
    

    
# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
# plt.legend(handlelength=3, bbox_to_anchor=(1.57, .8),loc='right')
plt.legend(handlelength=3, loc='upper right')
plt.subplot(1, 2, 2)

color_adj_start1 = 1
color_adj_start2 = 1
color_increment1 = -.4
color_increment2 = -.2
# Noise Figure for loop with RIN sweep
for i in range(len(LNA_NF_sweep)):
    if i == 0: 
        plt.plot(Pin,SFDR_EM_LNA[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label=label_EM+label_LNA)
#         plt.plot(Pin,SFDR_DML_LNA[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label=label_DM+label_LNA)
    else: 
        plt.plot(Pin,SFDR_EM_LNA[i,:], color = adjust_lightness(color_EM,color_adj_start1-i*color_increment1), linestyle = line_EM, linewidth=line_w,label='_nolegend_')
#         plt.plot(Pin,SFDR_DML_LNA[i,:], color = adjust_lightness(color_DM,color_adj_start2-i*color_increment2), linestyle = line_DM, linewidth=line_w,label='_nolegend_')

plt.plot(Pin,SFDR_3, color = 'k', linestyle = line_EM, linewidth=line_w-0.5,label=label_EM+'w/o LNA')
# plt.plot(Pin,SFDR_3DML, color = 'k', linestyle = line_DM, linewidth=line_w-0.5,label=label_DM)

# plt.vlines(10,125,110, color ='k', linestyles='dashed',linewidth=line_w+1)
# plt.text(0.5, 104, 'Small Signal Modulation\n Region of Direct Mod.', fontsize = 12)
# plt.arrow(9.5, 112, -3, 0, width=0.5, shape="full", head_length = 0.3, head_width = 2, color = 'k')

# text_index = 10
text_index2 = 14
if place_text:
#     plt.text(text_index-1.5, 101, best_sld_str, fontsize = 12)
#     plt.text(text_index-1.5, 92, worst_sld_str, fontsize = 12)
#     plt.arrow(text_index, 100.5, 0, -5.5, width=0.05, shape="full", head_length = 0.5, head_width = 0.5, color = 'k',zorder=40)

    plt.text(text_index2-1, 88.3, best_LNA_NF_str, fontsize = 12)
    plt.text(text_index2-1, 87.2, worst_LNA_NF_str, fontsize = 12)
    plt.arrow(text_index2,88.25, 0, -.8, width=0.05, shape="full", head_length = 0.1, head_width = 0.3, color = 'k',zorder=40)
    
    
plt.ylabel('Spurious-Free Dynamic Range (dB/Hz^2/3)', fontsize=fs-2)
plt.xlabel('Optical Input Power (dBm)', fontsize=fs)
# plt.title('RF Link Loss of Rx path for Canceller Architectures ', fontsize=15)
plt.xlim(8,20)
plt.ylim(86,88.5)

# plt.gca().invert_yaxis()
plt.xticks(fontsize=fs-2)
plt.yticks(fontsize=fs-2)
plt.grid(b=None, which='major', axis='both')
# plt.legend(handlelength=3, bbox_to_anchor=(1.57, .8),loc='right')
plt.legend(handlelength=3, loc='best')
if isSaving:
    outputFile = 'TFig2727b.pdf'
    plt.savefig(outputFile, format='pdf', dpi=dpi_res,bbox_inches='tight')
plt.show()

# See Examples from Hybrid Paper and RF analysis of PNN

# Example Data

In [ ]:
isLoading = True
import pickle

# Just want an example link from 0 to 10 GHz for link gain
if isLoading:
    S21 = []
    freq = []
    i = 0 
    mA = np.linspace(0,2.5, 11)
    inputFile = './Tuned_Cancellation_v2_24.11.2022.data'
    fd = open(inputFile, 'rb')
    dataset = pickle.load(fd)
    S21 = np.zeros((11,len(dataset[0])))
    S21 = dataset[0] #dBm
    freq = dataset[1] #freq
    Rx_S21 = dataset[2] #dBm
    Rx_freq = dataset[3] #freq
    
    colors = plt.cm.magma(np.linspace(0.3, 0.8, len(S21)))
    plt.figure(figsize=(8, 6))
    for i in range(len(S21)):
        plt.plot(Rx_freq[0:500]*1e-9,Rx_S21[0:500], color='r',linewidth=line_w)
        plt.ylabel(' Link Loss (dB)',fontsize=fs+4)
        plt.xlabel('Frequency (GHz)',fontsize=fs+4)
        plt.xticks(fontsize=fs)
        plt.yticks(fontsize=fs)
        plt.gca().invert_yaxis()
        plt.title('')
        plt.xlim(0.5,10)
        plt.grid(True)
    if isSaving:
        outputFile = 'TFig32_ExamplePhotonicLink.pdf'
        plt.savefig(outputFile, format='pdf', dpi=dpi_res,bbox_inches='tight')
    plt.show()



